# Plan
- Objective: Achieve medal-level AUC-ROC on TPS May 2022 via strong GBDT baseline + sensible CV + iterative feature engineering.
- Steps:
  1) Load data, basic EDA (shapes, dtypes, target distribution).
  2) Baseline model: LightGBM with StratifiedKFold CV, robust parameters, early stopping, logging.
  3) Generate out-of-fold (OOF) predictions, compute CV AUC, predict test, save submission.csv.
  4) Iterate: try interaction features (pairwise products/sums), categorical-like encodings if applicable, and alternative models (CatBoost/XGBoost), blending.
  5) Hyperparam refinements guided by CV; ensure leak-free processing.

Note: Keep cells modular; log timing per fold; use seed control for reproducibility.

Medal targets:
- Bronze: AUC >= 0.99818; Stretch: >= 0.99822.

Next: Implement baseline pipeline.

In [1]:
# Imports, versions, and utils
import os, sys, gc, time, random, math, json
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

SEED = 42
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(SEED)

print('Python', sys.version)
print('Pandas', pd.__version__)
print('Numpy', np.__version__)

def log(msg):
    ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'[{ts}] {msg}', flush=True)

Python 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
Pandas 2.2.2
Numpy 1.26.4


In [2]:
# Load data, basic EDA, dtype optimization, f_27 FE, and 10-fold splits
log('Loading data...')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
log(f'train shape: {train.shape}, test shape: {test.shape}')

# Basic target info
log('Target distribution:')
log(train['target'].value_counts(normalize=True).to_dict())

# Identify columns
id_col = 'id' if 'id' in train.columns else None
target_col = 'target'
all_cols = train.columns.tolist()
num_cols = [c for c in all_cols if c not in [id_col, target_col, 'f_27'] and train[c].dtype != 'object']

# Cast numerics to float32 for memory
for c in num_cols:
    train[c] = train[c].astype('float32')
    if c in test.columns:
        test[c] = test[c].astype('float32')
log('Numeric columns cast to float32.')

# f_27 feature engineering
assert 'f_27' in train.columns, 'f_27 not found'
all_f27 = pd.concat([train['f_27'], test['f_27']], axis=0).astype(str)
chars = sorted(set(''.join(all_f27.values)))
char2int = {ch:i for i, ch in enumerate(chars)}
log(f'f_27 unique chars: {len(chars)} -> {chars[:20]}...')

def add_f27_features(df):
    s = df['f_27'].astype(str)
    # positional features (10 positions)
    for i in range(10):
        col = f'f_27_pos_{i}'
        df[col] = s.str[i].map(char2int).astype('int8')
    # inverse-position features for A..J (find returns -1..9); store as int8 with offset
    for ch in list('ABCDEFGHIJ'):
        col = f'f_27_char_{ch}_pos'
        df[col] = s.str.find(ch).astype('int8')
    return df

t0 = time.time()
train = add_f27_features(train)
test = add_f27_features(test)
log(f'Added f_27 features in {time.time()-t0:.2f}s')

# Optional cheap row stats
row_num_cols = num_cols.copy()
if len(row_num_cols) > 0:
    train['row_sum'] = train[row_num_cols].sum(axis=1).astype('float32')
    train['row_std'] = train[row_num_cols].std(axis=1).astype('float32')
    test['row_sum'] = test[row_num_cols].sum(axis=1).astype('float32')
    test['row_std'] = test[row_num_cols].std(axis=1).astype('float32')
    log('Added row_sum and row_std')

# Update feature list after FE
fe_cols = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
log(f'Feature count (after FE): {len(fe_cols)}')

# Check nulls
null_train = train[fe_cols + [target_col]].isnull().sum().sum()
null_test = test[[c for c in fe_cols if c in test.columns]].isnull().sum().sum()
log(f'Nulls -> train: {null_train}, test: {null_test}')

# Save 10-fold stratified splits
log('Creating StratifiedKFold(10) indices...')
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
folds = np.full(len(train), -1, dtype=np.int16)
for fold, (_, val_idx) in enumerate(skf.split(train[fe_cols], train[target_col])):
    folds[val_idx] = fold
assert (folds >= 0).all()

folds_df = pd.DataFrame({id_col if id_col else 'row_id': train[id_col] if id_col else np.arange(len(train)), 'fold': folds})
folds_path = 'folds_10fold_seed42.csv'
folds_df.to_csv(folds_path, index=False)
log(f'Saved folds to {folds_path}')

# Save memory
gc.collect();
log('Data prep done.')

[2025-09-11 22:14:15] Loading data...


[2025-09-11 22:14:20] train shape: (800000, 33), test shape: (100000, 32)


[2025-09-11 22:14:20] Target distribution:


[2025-09-11 22:14:20] {0: 0.51386, 1: 0.48614}


[2025-09-11 22:14:20] Numeric columns cast to float32.


...

[2025-09-11 22:14:28] Feature count (after FE): 52


[2025-09-11 22:14:28] Nulls -> train: 0, test: 0


[2025-09-11 22:14:28] Creating StratifiedKFold(10) indices...


[2025-09-11 22:14:29] Saved folds to folds_10fold_seed42.csv


[2025-09-11 22:14:29] Data prep done.


In [ ]:
# LightGBM 10-fold CV training, OOF/test preds, submission
t_start = time.time()
log('Setting up LightGBM training...')
try:
    import lightgbm as lgb
except ImportError:
    import sys, subprocess
    log('LightGBM not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'lightgbm'])
    import lightgbm as lgb

# Features and target
features = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
X = train[features]
y = train[target_col].values
X_test = test[features]

# Folds
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values

# Params
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 256,
    'max_depth': -1,
    'min_data_in_leaf': 150,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.0,
    'lambda_l2': 2.0,
    'max_bin': 255,
    'verbose': -1,
    'n_jobs': -1,
    'seed': SEED
}

n_splits = len(np.unique(folds_arr))
oof = np.zeros(len(train), dtype='float32')
pred_test = np.zeros(len(test), dtype='float32')
feat_imps = []

for fold in range(n_splits):
    fold_t0 = time.time()
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')

    dtrain = lgb.Dataset(X.iloc[trn_idx], label=y[trn_idx])
    dvalid = lgb.Dataset(X.iloc[val_idx], label=y[val_idx])

    model = lgb.train(
        params,
        dtrain,
        num_boost_round=10000,
        valid_sets=[dtrain, dvalid],
        valid_names=['train','valid'],
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(200)]
    )

    oof[val_idx] = model.predict(X.iloc[val_idx], num_iteration=model.best_iteration)
    fold_auc = roc_auc_score(y[val_idx], oof[val_idx])
    log(f'Fold {fold} AUC: {fold_auc:.6f} | best_iter={model.best_iteration} | elapsed={time.time()-fold_t0:.1f}s')

    pred_test += model.predict(X_test, num_iteration=model.best_iteration) / n_splits

    fi = pd.DataFrame({'feature': features, f'fold_{fold}': model.feature_importance(importance_type='gain')})
    feat_imps.append(fi)
    del dtrain, dvalid, model
    gc.collect()

cv_auc = roc_auc_score(y, oof)
log(f'OOF AUC: {cv_auc:.6f}')

# Save OOF and test preds
pd.DataFrame({id_col: train[id_col], 'oof': oof}).to_csv('oof_lgb_seed42.csv', index=False)
pd.DataFrame({id_col: test[id_col], 'prediction': pred_test}).to_csv('pred_lgb_seed42.csv', index=False)

# Feature importance
fi_merged = feat_imps[0]
for i in range(1, len(feat_imps)):
    fi_merged = fi_merged.merge(feat_imps[i], on='feature', how='outer')
fi_merged.fillna(0, inplace=True)
fi_merged['gain_mean'] = fi_merged[[c for c in fi_merged.columns if c.startswith('fold_')]].mean(axis=1)
fi_merged.sort_values('gain_mean', ascending=False).to_csv('feature_importance_lgb.csv', index=False)
log('Saved OOF, test preds, and feature importance.')

# Submission
sub = pd.read_csv('sample_submission.csv')
sub['target'] = pred_test
sub.to_csv('submission.csv', index=False)
log(f'Submission saved. Total training time: {time.time()-t_start:.1f}s')

In [3]:
# Patch f_27 features (+1 offset), add bigrams/runs/equality/char counts; add extra row stats
log('Patching/adding f_27 features and cheap row stats (queued for after current run)')

# Ensure we have shared char2int and strings
all_f27 = pd.concat([train['f_27'], test['f_27']], axis=0).astype(str)
chars = sorted(set(''.join(all_f27.values)))  # expecting 20 A..T
char2int = {ch:i for i, ch in enumerate(chars)}

def build_bigrams_map(series):
    # Build bigram vocabulary from train+test
    seen = set()
    for s in series.values:
        for i in range(9):
            seen.add(s[i:i+2])
    bigrams = sorted(seen)
    return {bg:i for i, bg in enumerate(bigrams)}

bigrams2int = build_bigrams_map(all_f27)
log(f'Bigram vocab size: {len(bigrams2int)}')

def longest_run_and_transitions(s):
    # s length is 10
    max_run = 1
    cur = 1
    transitions = 0
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            cur += 1
        else:
            transitions += 1
            if cur > max_run:
                max_run = cur
            cur = 1
    if cur > max_run:
        max_run = cur
    return max_run, transitions

def add_extra_f27_features(df):
    s = df['f_27'].astype(str)
    # Offset existing find features by +1 (range 0..10); and presence flag
    for ch in list('ABCDEFGHIJ'):
        col = f'f_27_char_{ch}_pos'
        if col in df.columns:
            df[col] = (df[col].astype('int16') + 1).astype('int8')
        pres = f'f_27_char_{ch}_present'
        df[pres] = (s.str.find(ch) >= 0).astype('int8')
    # Adjacent equality flags pos0==pos1 ... pos8==pos9
    for i in range(9):
        df[f'f_27_adj_eq_{i}_{i+1}'] = (s.str[i] == s.str[i+1]).astype('int8')
    # 9 bigram columns: s[i:i+2] label-encoded
    for i in range(9):
        col = f'f_27_bg_{i}_{i+1}'
        df[col] = s.str[i:i+2].map(bigrams2int).astype('int16')
    # longest_run_length and transitions_count
    lr, tr = zip(*s.map(longest_run_and_transitions))
    df['f_27_longest_run'] = np.array(lr, dtype='int8')
    df['f_27_transitions'] = np.array(tr, dtype='int8')
    # Per-char counts A..T (20 ints)
    for ch in chars:
        df[f'f_27_cnt_{ch}'] = s.str.count(ch).astype('int8')
    return df

t0 = time.time()
train = add_extra_f27_features(train)
test = add_extra_f27_features(test)
log(f'Added extra f_27 features in {time.time()-t0:.2f}s')

# Cheap row stats additions on numeric block
row_num_cols = [c for c in train.columns if c not in [id_col, target_col, 'f_27'] and c.startswith('f_') and train[c].dtype in [np.float32, np.float64, 'float32', 'float64'] ]
if len(row_num_cols) > 0:
    train['row_min'] = train[row_num_cols].min(axis=1).astype('float32')
    train['row_max'] = train[row_num_cols].max(axis=1).astype('float32')
    train['row_mean'] = train[row_num_cols].mean(axis=1).astype('float32')
    test['row_min'] = test[row_num_cols].min(axis=1).astype('float32')
    test['row_max'] = test[row_num_cols].max(axis=1).astype('float32')
    test['row_mean'] = test[row_num_cols].mean(axis=1).astype('float32')
    train['row_max_minus_mean'] = (train['row_max'] - train['row_mean']).astype('float32')
    train['mean_minus_row_min'] = (train['row_mean'] - train['row_min']).astype('float32')
    test['row_max_minus_mean'] = (test['row_max'] - test['row_mean']).astype('float32')
    test['mean_minus_row_min'] = (test['row_mean'] - test['row_min']).astype('float32')
    log('Added row_min/row_max/row_mean and z-extremes')

# Refresh feature list (exclude id/target/raw f_27)
fe_cols = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
log(f'Feature count after patch: {len(fe_cols)}')
gc.collect();

[2025-09-11 22:14:35] Patching/adding f_27 features and cheap row stats (queued for after current run)


[2025-09-11 22:14:37] Bigram vocab size: 349


/tmp/ipykernel_400/269042634.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'f_27_cnt_{ch}'] = s.str.count(ch).astype('int8')


/tmp/ipykernel_400/269042634.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'f_27_cnt_{ch}'] = s.str.count(ch).astype('int8')


/tmp/ipykernel_400/269042634.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'f_27_cnt_{ch}'] = s.str.count(ch).astype('int8')


...

/tmp/ipykernel_400/269042634.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['row_min'] = train[row_num_cols].min(axis=1).astype('float32')


/tmp/ipykernel_400/269042634.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['row_max'] = train[row_num_cols].max(axis=1).astype('float32')


[2025-09-11 22:14:57] Added row_min/row_max/row_mean and z-extremes


[2025-09-11 22:14:57] Feature count after patch: 107


/tmp/ipykernel_400/269042634.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['row_mean'] = train[row_num_cols].mean(axis=1).astype('float32')
/tmp/ipykernel_400/269042634.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['row_min'] = test[row_num_cols].min(axis=1).astype('float32')
/tmp/ipykernel_400/269042634.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.con

In [4]:
# Add targeted numeric interactions and quick-win f_27 features (vectorized, cheap)
log('Adding numeric interactions and f_27 quick wins...')

t0 = time.time()

# Helper: get col safely
def _col(name):
    if name not in train.columns or name not in test.columns:
        raise KeyError(f'Missing column: {name}')
    return name

# Base numeric columns expected in TPS May 2022
base_cols = ['f_00','f_01','f_02','f_03','f_05','f_06','f_10','f_12','f_20','f_21','f_22','f_26']
for c in base_cols:
    _ = _col(c)

def build_num_interactions(df):
    eps = 1e-6
    out = {}
    f00 = df['f_00'].astype('float32').values
    f01 = df['f_01'].astype('float32').values
    f02 = df['f_02'].astype('float32').values
    f03 = df['f_03'].astype('float32').values
    f05 = df['f_05'].astype('float32').values
    f06 = df['f_06'].astype('float32').values
    f10 = df['f_10'].astype('float32').values
    f12 = df['f_12'].astype('float32').values
    f20 = df['f_20'].astype('float32').values
    f21 = df['f_21'].astype('float32').values
    f22 = df['f_22'].astype('float32').values
    f26 = df['f_26'].astype('float32').values
    # Ratios
    out['int_ratio_00_01'] = (f00 / (f01 + eps)).astype('float32')
    out['int_ratio_02_03'] = (f02 / (f03 + eps)).astype('float32')
    out['int_ratio_10_12'] = (f10 / (f12 + eps)).astype('float32')
    out['int_ratio_20_21'] = (f20 / (f21 + eps)).astype('float32')
    out['int_ratio_21_02'] = (f21 / (f02 + eps)).astype('float32')
    out['int_ratio_22_02'] = (f22 / (f02 + eps)).astype('float32')
    out['int_ratio_05_06'] = (f05 / (f06 + eps)).astype('float32')
    out['int_ratio_26_02'] = (f26 / (f02 + eps)).astype('float32')
    # Products
    out['int_prod_00_10'] = (f00 * f10).astype('float32')
    out['int_prod_02_20'] = (f02 * f20).astype('float32')
    out['int_prod_01_21'] = (f01 * f21).astype('float32')
    out['int_prod_21_02'] = (f21 * f02).astype('float32')
    out['int_prod_22_05'] = (f22 * f05).astype('float32')
    # Diffs
    out['int_absdiff_00_10'] = np.abs(f00 - f10).astype('float32')
    out['int_absdiff_02_20'] = np.abs(f02 - f20).astype('float32')
    out['int_absdiff_01_21'] = np.abs(f01 - f21).astype('float32')
    out['int_absdiff_21_02'] = np.abs(f21 - f02).astype('float32')
    out['int_diff_22_05'] = (f22 - f05).astype('float32')
    out['int_diff_26_00'] = (f26 - f00).astype('float32')
    # Squares
    out['int_sq_00'] = (f00 * f00).astype('float32')
    out['int_sq_02'] = (f02 * f02).astype('float32')
    out['int_sq_10'] = (f10 * f10).astype('float32')
    out['int_sq_20'] = (f20 * f20).astype('float32')
    return pd.DataFrame(out)

num_int_train = build_num_interactions(train)
num_int_test = build_num_interactions(test)

# f_27 quick wins using positional ints and counts already present
pos_cols = [f'f_27_pos_{i}' for i in range(10)]
for c in pos_cols:
    if c not in train.columns:
        raise KeyError(f'Missing {c} for f_27 quick features')

# Build numpy arrays for positions
P_tr = np.stack([train[c].astype('int16').values for c in pos_cols], axis=1)  # (n,10)
P_te = np.stack([test[c].astype('int16').values for c in pos_cols], axis=1)

# Per-char counts A..T exist as f_27_cnt_{ch}; assemble count matrices if present, else compute from positions
chars = sorted(set(''.join(pd.concat([train['f_27'], test['f_27']]).astype(str).values)))
char2int = {ch:i for i,ch in enumerate(chars)}
vocab_size = len(chars)  # expected 20

def counts_from_positions(P, vocab_size):
    # P: (n,10) with 0..vocab_size-1
    n = P.shape[0]
    cnt = np.zeros((n, vocab_size), dtype=np.int16)
    for k in range(10):
        idx = P[:, k]
        # bincount per row via advanced indexing
        cnt[np.arange(n), idx] += 1
    return cnt

# Try to build counts from existing columns if available
cnt_cols = [c for c in train.columns if c.startswith('f_27_cnt_')]
if len(cnt_cols) == vocab_size:
    CNT_tr = np.stack([train[c].astype('int16').values for c in cnt_cols], axis=1)
    CNT_te = np.stack([test[c].astype('int16').values for c in cnt_cols], axis=1)
else:
    CNT_tr = counts_from_positions(P_tr, vocab_size)
    CNT_te = counts_from_positions(P_te, vocab_size)

def build_f27_quick(P, CNT):
    n = P.shape[0]
    out = {}
    # nunique
    out['f27_nunique'] = (CNT > 0).sum(axis=1).astype('int8')
    # equal pairs per row: sum cnt*(cnt-1)/2
    cnt = CNT.astype('int32')
    out['f27_equal_pairs'] = ((cnt * (cnt - 1)) // 2).sum(axis=1).astype('int32')
    # entropy over 10 positions
    p = cnt / 10.0
    with np.errstate(divide='ignore', invalid='ignore'):
        ent = -(p * np.log(p + 1e-12)).sum(axis=1)
    out['f27_entropy'] = ent.astype('float32')
    # first-last same
    out['f27_first_last_same'] = (P[:, 0] == P[:, 9]).astype('int8')
    # palindrome matches
    pal = np.zeros(n, dtype=np.int8)
    for i in range(5):
        pal += (P[:, i] == P[:, 9 - i]).astype('int8')
    out['f27_pal_matches'] = pal
    # circular shift equal counts for k=1,2,3
    for k in (1, 2, 3):
        eq = (P == np.roll(P, shift=-k, axis=1)).sum(axis=1).astype('int8')
        out[f'f27_circ_eq_k{k}'] = eq
    # num_runs, mean_run_length, transitions_parity using existing transitions if present
    if 'f_27_transitions' in train.columns:
        trn = None  # placeholder
    # compute transitions from positions
    trans = (P[:, 1:] != P[:, :-1]).sum(axis=1).astype('int8')
    out['f27_num_runs'] = (trans + 1).astype('int8')
    out['f27_mean_run_len'] = (10.0 / (trans + 1).clip(min=1)).astype('float32')
    out['f27_transitions_parity'] = (trans % 2).astype('int8')
    # 3-gram hashed (base-23 hash mod 512) for i=0..7
    base = 23
    mod = 512
    tri_feats = {}
    for i in range(8):
        code = (P[:, i] * (base * base) + P[:, i + 1] * base + P[:, i + 2]) % mod
        tri_feats[f'f27_tri_hash_{i}_{i+2}'] = code.astype('int16')
    out.update(tri_feats)
    return pd.DataFrame(out)

f27q_train = build_f27_quick(P_tr, CNT_tr)
f27q_test = build_f27_quick(P_te, CNT_te)

# Concatenate blocks to reduce fragmentation
train = pd.concat([train, num_int_train, f27q_train], axis=1)
test = pd.concat([test, num_int_test, f27q_test], axis=1)

log(f'Added interactions and f27 quick features in {time.time()-t0:.1f}s | new feature count: {train.shape[1]-3}')
gc.collect();

[2025-09-11 22:15:07] Adding numeric interactions and f_27 quick wins...


[2025-09-11 22:15:08] Added interactions and f27 quick features in 1.4s | new feature count: 149


In [5]:
# CatBoost 10-fold CV training, OOF/test preds, for blending
log('Setting up CatBoost training (same folds/features, fast params)...')
try:
    from catboost import CatBoostClassifier, Pool
except ImportError:
    import sys, subprocess
    log('CatBoost not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'catboost'])
    from catboost import CatBoostClassifier, Pool

# Reuse features from current dataframe
features = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
X = train[features]
y = train[target_col].values
X_test = test[features]

# Identify categorical-like features (int-coded f_27-derived columns)
cat_cols = [
    c for c in features
    if (c.startswith('f_27_pos_') or c.startswith('f_27_char_') or c.startswith('f_27_bg_')
        or c.endswith('_present') or c.startswith('f_27_adj_eq_') or c.startswith('f27_tri_hash_'))
]
cat_idx = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]
log(f'CatBoost categorical features count: {len(cat_idx)}')

# Folds
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

# Fast strong params (GPU) per expert guidance
params = {
    'loss_function': 'Logloss', 'eval_metric': 'AUC',
    'iterations': 3800, 'learning_rate': 0.032, 'depth': 8,
    'l2_leaf_reg': 6.0, 'bootstrap_type': 'Poisson', 'subsample': 0.82,
    'sampling_frequency': 'PerTree', 'border_count': 128,
    'grow_policy': 'SymmetricTree', 'max_ctr_complexity': 1,
    'od_type': 'Iter', 'od_wait': 120,
    'task_type': 'GPU', 'devices': '0',
    'random_seed': 42, 'verbose': 400,
    # Ensure no GPU-incompatible column sampling (rsm) is used
    'rsm': 1.0
}

oof_cb = np.zeros(len(train), dtype='float32')
pred_test_cb = np.zeros(len(test), dtype='float32')

for fold in range(n_splits):
    fold_t0 = time.time()
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[CB] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')

    train_pool = Pool(X.iloc[trn_idx], y[trn_idx], cat_features=cat_idx)
    valid_pool = Pool(X.iloc[val_idx], y[val_idx], cat_features=cat_idx)

    model_cb = CatBoostClassifier(**params)
    model_cb.fit(train_pool, eval_set=valid_pool, use_best_model=True)

    oof_cb[val_idx] = model_cb.predict_proba(valid_pool)[:,1].astype('float32')
    fold_auc = roc_auc_score(y[val_idx], oof_cb[val_idx])
    log(f'[CB] Fold {fold} AUC: {fold_auc:.6f} | best_iter={model_cb.tree_count_} | elapsed={time.time()-fold_t0:.1f}s')

    pred_test_cb += model_cb.predict_proba(Pool(X_test, cat_features=cat_idx))[:,1].astype('float32') / n_splits
    del model_cb, train_pool, valid_pool
    gc.collect()

cv_auc_cb = roc_auc_score(y, oof_cb)
log(f'[CB] OOF AUC: {cv_auc_cb:.6f}')

# Save OOF and test preds
pd.DataFrame({id_col: train[id_col], 'oof_cb': oof_cb}).to_csv('oof_catboost_seed42.csv', index=False)
pd.DataFrame({id_col: test[id_col], 'prediction_cb': pred_test_cb}).to_csv('pred_catboost_seed42.csv', index=False)
log('[CB] Saved OOF and test predictions.')

[2025-09-11 22:15:16] Setting up CatBoost training (same folds/features, fast params)...


[2025-09-11 22:15:16] CatBoost categorical features count: 56


[2025-09-11 22:15:16] [CB] Fold 1/10 | trn=720000 val=80000


CatBoostError: catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 100: no CUDA-capable device is detected

In [7]:
# LightGBM fast 10-fold CV (numeric-only), with early stopping and faster params
t_start = time.time()
log('Setting up LightGBM (fast params) training...')
import lightgbm as lgb

# Features and target (after latest FE, exclude id/target/raw f_27)
features = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
X = train[features]
y = train[target_col].values
X_test = test[features]

# Folds
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

# Fast params per expert guidance
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.045,
    'num_leaves': 144,
    'max_depth': -1,
    'min_data_in_leaf': 240,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'lambda_l1': 0.0,
    'lambda_l2': 4.0,
    'max_bin': 127,
    'verbose': -1,
    'n_jobs': -1,
    'seed': SEED,
    'feature_fraction_seed': SEED,
    'bagging_seed': SEED,
    'data_random_seed': SEED
}

oof_fast = np.zeros(len(train), dtype='float32')
pred_test_fast = np.zeros(len(test), dtype='float32')
feat_imps = []

for fold in range(n_splits):
    fold_t0 = time.time()
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[LGB fast] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')

    dtrain = lgb.Dataset(X.iloc[trn_idx], label=y[trn_idx])
    dvalid = lgb.Dataset(X.iloc[val_idx], label=y[val_idx])

    model = lgb.train(
        params,
        dtrain,
        num_boost_round=4000,
        valid_sets=[dtrain, dvalid],
        valid_names=['train','valid'],
        callbacks=[lgb.early_stopping(120), lgb.log_evaluation(100)]
    )

    oof_fast[val_idx] = model.predict(X.iloc[val_idx], num_iteration=model.best_iteration)
    fold_auc = roc_auc_score(y[val_idx], oof_fast[val_idx])
    log(f'[LGB fast] Fold {fold} AUC: {fold_auc:.6f} | best_iter={model.best_iteration} | elapsed={time.time()-fold_t0:.1f}s')

    pred_test_fast += model.predict(X_test, num_iteration=model.best_iteration) / n_splits

    fi = pd.DataFrame({'feature': features, f'fold_{fold}': model.feature_importance(importance_type='gain')})
    feat_imps.append(fi)
    del dtrain, dvalid, model
    gc.collect()

cv_auc_fast = roc_auc_score(y, oof_fast)
log(f'[LGB fast] OOF AUC: {cv_auc_fast:.6f}')

# Save OOF and test preds
pd.DataFrame({id_col: train[id_col], 'oof_lgb_fast': oof_fast}).to_csv('oof_lgb_fast_seed42.csv', index=False)
pd.DataFrame({id_col: test[id_col], 'prediction_lgb_fast': pred_test_fast}).to_csv('pred_lgb_fast_seed42.csv', index=False)

# Feature importance
fi_merged = feat_imps[0]
for i in range(1, len(feat_imps)):
    fi_merged = fi_merged.merge(feat_imps[i], on='feature', how='outer')
fi_merged.fillna(0, inplace=True)
fi_merged['gain_mean'] = fi_merged[[c for c in fi_merged.columns if c.startswith('fold_')]].mean(axis=1)
fi_merged.sort_values('gain_mean', ascending=False).to_csv('feature_importance_lgb_fast.csv', index=False)
log('[LGB fast] Saved OOF, test preds, and feature importance.')

# Submission
sub = pd.read_csv('sample_submission.csv')
sub['target'] = pred_test_fast
sub.to_csv('submission_lgb_fast.csv', index=False)
log(f'[LGB fast] Submission saved. Total training time: {time.time()-t_start:.1f}s')

[2025-09-11 22:16:24] Setting up LightGBM (fast params) training...


[2025-09-11 22:16:25] [LGB fast] Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 120 rounds


[100]	train's auc: 0.963129	valid's auc: 0.960292


[200]	train's auc: 0.978775	valid's auc: 0.975702


...

[3100]	train's auc: 0.999896	valid's auc: 0.99081


[3200]	train's auc: 0.999919	valid's auc: 0.990819


[3300]	train's auc: 0.999937	valid's auc: 0.990833


[3400]	train's auc: 0.999951	valid's auc: 0.99086


[3500]	train's auc: 0.999962	valid's auc: 0.990883


In [ ]:
# Blend OOF to optimize weights; produce blended submission
log('Setting up blending between LGB fast and CatBoost...')
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

def load_oof_pred(oof_path, col_name):
    df = pd.read_csv(oof_path)
    if col_name not in df.columns:
        # assume last column is the prediction
        pred_col = [c for c in df.columns if c != id_col][0]
    else:
        pred_col = col_name
    return df[[id_col, pred_col]].rename(columns={pred_col: 'pred'})

def load_test_pred(pred_path, col_name):
    df = pd.read_csv(pred_path)
    if col_name not in df.columns:
        pred_col = [c for c in df.columns if c != id_col][0]
    else:
        pred_col = col_name
    return df[[id_col, pred_col]].rename(columns={pred_col: 'pred'})

oof_lgb_path = 'oof_lgb_fast_seed42.csv'
pred_lgb_path = 'pred_lgb_fast_seed42.csv'
oof_cb_path = 'oof_catboost_seed42.csv'
pred_cb_path = 'pred_catboost_seed42.csv'

# Wait-safe loads (one may not exist yet)
oof_lgb = load_oof_pred(oof_lgb_path, 'oof_lgb_fast') if os.path.exists(oof_lgb_path) else None
pred_lgb = load_test_pred(pred_lgb_path, 'prediction_lgb_fast') if os.path.exists(pred_lgb_path) else None
oof_cb = load_oof_pred(oof_cb_path, 'oof_cb') if os.path.exists(oof_cb_path) else None
pred_cb = load_test_pred(pred_cb_path, 'prediction_cb') if os.path.exists(pred_cb_path) else None

if (oof_lgb is None) or (pred_lgb is None):
    log('LGB predictions not found yet; run LGB first.')
else:
    # Merge with ground truth
    gt = train[[id_col, target_col]].copy()
    oof = gt.merge(oof_lgb, on=id_col, how='left').rename(columns={'pred': 'lgb'})
    have_cb = (oof_cb is not None) and (pred_cb is not None)
    if have_cb:
        oof = oof.merge(oof_cb.rename(columns={'pred': 'cb'}), on=id_col, how='left')
    else:
        oof['cb'] = np.nan

    # Rank transform helper
    def rank_norm(x):
        r = pd.Series(x).rank(method='average').values
        return (r - 1) / (len(r) - 1)

    best_auc = 0.0
    best_w = None
    oof['lgb_r'] = rank_norm(oof['lgb'])
    if have_cb:
        oof['cb_r'] = rank_norm(oof['cb'])

    # Grid search weights (0..1, step 0.05). If CB missing, just evaluate LGB.
    weights = np.linspace(0.0, 1.0, 21) if have_cb else [1.0]
    for w in weights:
        if have_cb:
            blend = w * oof['lgb'] + (1 - w) * oof['cb']
            blend_r = w * oof['lgb_r'] + (1 - w) * oof['cb_r']
        else:
            blend = oof['lgb']
            blend_r = oof['lgb_r']
        auc = roc_auc_score(oof[target_col], blend)
        auc_r = roc_auc_score(oof[target_col], blend_r)
        if auc > best_auc:
            best_auc = auc
            best_w = ('prob', float(w))
            best_mode = 'prob'
            best_is_rank = False
            best_auc_r = auc_r
        if auc_r > best_auc:
            best_auc = auc_r
            best_w = ('rank', float(w))
            best_mode = 'rank'
            best_is_rank = True
            best_auc_r = auc_r

    log(f"Blending search done. Best OOF AUC={best_auc:.6f} | mode={best_mode} | w_lgb={best_w[1] if best_w else 1.0}")

    # Build test blend
    sub = pd.read_csv('sample_submission.csv')
    test_merge = sub[[id_col]].merge(pred_lgb.rename(columns={'pred': 'lgb'}), on=id_col, how='left')
    if have_cb:
        test_merge = test_merge.merge(pred_cb.rename(columns={'pred': 'cb'}), on=id_col, how='left')
    else:
        test_merge['cb'] = np.nan

    if (best_w is None) or (not have_cb):
        # Fallback to LGB only
        sub['target'] = test_merge['lgb'].astype('float32')
        sub.to_csv('submission_blend.csv', index=False)
        sub.to_csv('submission.csv', index=False)
        log('Saved submission_blend.csv and submission.csv using LGB only (CB not available).')
    else:
        w = best_w[1]
        if best_mode == 'prob':
            pred_blend = w * test_merge['lgb'].values + (1 - w) * test_merge['cb'].values
        else:
            # rank-average
            rl = pd.Series(test_merge['lgb']).rank(method='average').values
            rc = pd.Series(test_merge['cb']).rank(method='average').values
            rl = (rl - 1) / (len(rl) - 1)
            rc = (rc - 1) / (len(rc) - 1)
            pred_blend = w * rl + (1 - w) * rc
        sub['target'] = pred_blend.astype('float32')
        sub.to_csv('submission_blend.csv', index=False)
        sub.to_csv('submission.csv', index=False)
        log('Saved submission_blend.csv and submission.csv with optimized blend.')

In [26]:
# Validate submission.csv format; fix if needed
log('Validating submission.csv...')
sub = pd.read_csv('submission.csv') if os.path.exists('submission.csv') else None
issues = []
if sub is None:
    issues.append('submission.csv missing')
else:
    # Basic checks
    if list(sub.columns) != ['id', 'target']:
        issues.append(f'Bad columns: {list(sub.columns)}')
    if len(sub) != len(test):
        issues.append(f'Row count mismatch: {len(sub)} vs test {len(test)}')
    if sub['target'].isna().any():
        issues.append(f'NaNs in target: {int(sub["target"].isna().sum())}')
    # dtype: ensure float
    try:
        sub['target'] = sub['target'].astype('float32')
    except Exception as e:
        issues.append(f'target casting error: {e}')

if issues:
    log('Issues found: ' + '; '.join(issues))
    # Rebuild from LGB fast preds directly
    log('Rebuilding submission.csv from pred_lgb_fast_seed42.csv...')
    pred_lgb = pd.read_csv('pred_lgb_fast_seed42.csv')
    # Align and sanity
    pred_lgb = pred_lgb.rename(columns={pred_lgb.columns[1]: 'prediction_lgb_fast'})
    ss = pd.read_csv('sample_submission.csv')
    merge = ss[['id']].merge(pred_lgb.rename(columns={'prediction_lgb_fast':'target'}), on='id', how='left')
    if merge['target'].isna().any():
        missing = int(merge['target'].isna().sum())
        raise ValueError(f'Missing predictions for {missing} test rows after rebuild.')
    merge['target'] = merge['target'].astype('float32')
    merge.to_csv('submission.csv', index=False)
    log('submission.csv rebuilt from LGB fast preds.')
else:
    log('submission.csv looks valid: ' + f"shape={sub.shape}, dtypes={dict(sub.dtypes)}")

[2025-09-12 02:03:09] Validating submission.csv...


[2025-09-12 02:03:09] submission.csv looks valid: shape=(100000, 2), dtypes={'id': dtype('int64'), 'target': dtype('float32')}


In [8]:
# Force-rebuild submission.csv strictly aligned to sample_submission; ensure finite [0,1] float64
log('Rebuilding submission.csv strictly to avoid format issues...')
ss = pd.read_csv('sample_submission.csv')
pred = pd.read_csv('pred_lgb_fast_seed42.csv')
# Identify prediction column (second column)
pred_col = [c for c in pred.columns if c != 'id']
assert len(pred_col) == 1, f'Unexpected pred columns: {pred.columns.tolist()}'
# Rename to avoid clash with ss.target during merge
pred = pred.rename(columns={pred_col[0]: 'prediction'})
# Left-merge to preserve order and count from sample_submission (use only id to avoid duplicate target columns)
sub = ss[['id']].merge(pred, on='id', how='left')
missing = sub['prediction'].isna().sum()
if missing > 0:
    raise ValueError(f'Missing predictions for {missing} test rows after strict rebuild.')
# Ensure numeric, finite, and in [0,1]
sub['target'] = pd.to_numeric(sub['prediction'], errors='coerce')
if sub['target'].isna().any():
    raise ValueError('NaNs found after numeric conversion.')
sub['target'] = sub['target'].astype('float64').clip(0.0, 1.0)
sub = sub[['id', 'target']]
# Final save
sub.to_csv('submission.csv', index=False)
log('Strict submission.csv rebuilt and saved: ' + f"shape={sub.shape}, dtypes={dict(sub.dtypes)}")

[2025-09-11 22:32:20] Rebuilding submission.csv strictly to avoid format issues...


[2025-09-11 22:32:21] Strict submission.csv rebuilt and saved: shape=(100000, 2), dtypes={'id': dtype('int64'), 'target': dtype('float64')}


In [6]:
# Minimal, high-impact FE block (sorted stats, equality count, parity, f27 majority)
log('Adding minimal high-impact features (sorted stats, equality count, parity, f27 majority)...')
t0 = time.time()

# 1) Row-wise numeric sorting stats
num_float_cols = [c for c in train.columns if (c not in [id_col, target_col, 'f_27']) and (str(train[c].dtype).startswith('float')) and c.startswith('f_')]
if len(num_float_cols) > 0:
    vals_tr = train[num_float_cols].values
    vals_te = test[num_float_cols].values
    srt_tr = np.sort(vals_tr, axis=1)
    srt_te = np.sort(vals_te, axis=1)
    train['f_sorted_0'] = srt_tr[:, 0].astype('float32')
    train['f_sorted_1'] = srt_tr[:, 1].astype('float32')
    train['f_sorted_-1'] = srt_tr[:, -1].astype('float32')
    train['f_sorted_range'] = (srt_tr[:, -1] - srt_tr[:, 0]).astype('float32')
    test['f_sorted_0'] = srt_te[:, 0].astype('float32')
    test['f_sorted_1'] = srt_te[:, 1].astype('float32')
    test['f_sorted_-1'] = srt_te[:, -1].astype('float32')
    test['f_sorted_range'] = (srt_te[:, -1] - srt_te[:, 0]).astype('float32')

# 2) Pairwise equality count over subset of strong numerics
num_list = ['f_00','f_01','f_02','f_03','f_05','f_06','f_10','f_12','f_20','f_21','f_22','f_26']
num_list = [c for c in num_list if c in train.columns and c in test.columns]
if len(num_list) >= 2:
    M_tr = train[num_list].values
    M_te = test[num_list].values
    from itertools import combinations
    def eq_pairs(M):
        n = M.shape[0]
        cnt = np.zeros(n, dtype=np.int16)
        for i, j in combinations(range(M.shape[1]), 2):
            cnt += (np.abs(M[:, i] - M[:, j]) < 1e-6)
        return cnt
    train['num_equal_pairs'] = eq_pairs(M_tr)
    test['num_equal_pairs'] = eq_pairs(M_te)

# 3) Quick parity/mod on a few columns
for c in ['f_00','f_02','f_10','f_20','f_21','f_26']:
    if c in train.columns and c in test.columns:
        train[f'{c}_parity'] = (train[c].astype('int64') & 1).astype('int8')
        test[f'{c}_parity'] = (test[c].astype('int64') & 1).astype('int8')

# 4) f_27 majority count and index (recompute counts from positions)
pos_cols = [f'f_27_pos_{i}' for i in range(10)]
if all(c in train.columns for c in pos_cols):
    P_tr = np.stack([train[c].astype('int16').values for c in pos_cols], axis=1)
    P_te = np.stack([test[c].astype('int16').values for c in pos_cols], axis=1)
    vocab_size = len(sorted(set(''.join(pd.concat([train['f_27'], test['f_27']]).astype(str).values))))
    def counts_from_positions(P, vocab_size):
        n = P.shape[0]
        CNT = np.zeros((n, vocab_size), dtype=np.int16)
        for k in range(P.shape[1]):
            CNT[np.arange(n), P[:, k]] += 1
        return CNT
    CNT_tr = counts_from_positions(P_tr, vocab_size)
    CNT_te = counts_from_positions(P_te, vocab_size)
    train['f27_majority_cnt'] = CNT_tr.max(axis=1).astype('int8')
    train['f27_majority_idx'] = CNT_tr.argmax(axis=1).astype('int8')
    test['f27_majority_cnt'] = CNT_te.max(axis=1).astype('int8')
    test['f27_majority_idx'] = CNT_te.argmax(axis=1).astype('int8')

log(f'Added minimal FE in {time.time()-t0:.2f}s | train cols={train.shape[1]}')
gc.collect();

[2025-09-11 22:16:17] Adding minimal high-impact features (sorted stats, equality count, parity, f27 majority)...


[2025-09-11 22:16:18] Added minimal FE in 1.25s | train cols=165


In [12]:
# XGBoost CPU 10-fold CV training, OOF/test preds, for blending (use xgb.train for broad version compatibility)
log('Setting up XGBoost (CPU, hist) training...')
t0_all = time.time()
try:
    import xgboost as xgb
except ImportError:
    import sys, subprocess
    log('XGBoost not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb

# Features/target (same 165-feature set; exclude id/target/raw f_27)
features = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
X = train[features]
y = train[target_col].values
X_test = test[features]

# Folds (locked)
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

# Params per expert guidance (CPU) mapped for xgb.train
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'max_bin': 128,
    'eta': 0.04,                 # learning_rate
    'max_depth': 8,
    'min_child_weight': 100,
    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'lambda': 3.0,               # reg_lambda
    'nthread': 36,
    'seed': 42
}
num_boost_round = 3000
early_stopping_rounds = 150

oof_xgb = np.zeros(len(train), dtype='float32')
pred_test_xgb = np.zeros(len(test), dtype='float32')

for fold in range(n_splits):
    fold_t0 = time.time()
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[XGB] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')

    dtrain = xgb.DMatrix(X.iloc[trn_idx], label=y[trn_idx])
    dvalid = xgb.DMatrix(X.iloc[val_idx], label=y[val_idx])
    dtest = xgb.DMatrix(X_test)

    bst = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=200
    )

    # Robust best iteration retrieval across versions
    best_iter = None
    attrs = {}
    try:
        attrs = bst.attributes()
    except Exception:
        attrs = {}
    if 'best_iteration' in attrs:
        try:
            best_iter = int(attrs['best_iteration'])
        except Exception:
            best_iter = None
    if (best_iter is None) and hasattr(bst, 'best_iteration') and (bst.best_iteration is not None):
        try:
            best_iter = int(bst.best_iteration)
        except Exception:
            best_iter = None

    # Predict using iteration_range when best_iter is available; otherwise use full model
    if best_iter is not None and best_iter >= 0:
        oof_pred = bst.predict(dvalid, iteration_range=(0, best_iter + 1))
    else:
        oof_pred = bst.predict(dvalid)
    oof_xgb[val_idx] = oof_pred.astype('float32')
    fold_auc = roc_auc_score(y[val_idx], oof_xgb[val_idx])
    log(f'[XGB] Fold {fold} AUC: {fold_auc:.6f} | best_iter={best_iter} | elapsed={time.time()-fold_t0:.1f}s')

    if best_iter is not None and best_iter >= 0:
        pred_te = bst.predict(dtest, iteration_range=(0, best_iter + 1))
    else:
        pred_te = bst.predict(dtest)
    pred_test_xgb += pred_te.astype('float32') / n_splits
    del dtrain, dvalid, dtest, bst
    gc.collect()

cv_auc_xgb = roc_auc_score(y, oof_xgb)
log(f'[XGB] OOF AUC: {cv_auc_xgb:.6f} | total_elapsed={time.time()-t0_all:.1f}s')

# Save artifacts
pd.DataFrame({id_col: train[id_col], 'oof_xgb': oof_xgb}).to_csv('oof_xgb_seed42.csv', index=False)
pd.DataFrame({id_col: test[id_col], 'prediction_xgb': pred_test_xgb}).to_csv('pred_xgb_seed42.csv', index=False)
log('[XGB] Saved OOF and test predictions.')

[2025-09-11 22:38:02] Setting up XGBoost (CPU, hist) training...


[2025-09-11 22:38:02] [XGB] Fold 1/10 | trn=720000 val=80000


[0]	train-auc:0.75379	valid-auc:0.75297


[200]	train-auc:0.96478	valid-auc:0.96191


[400]	train-auc:0.97630	valid-auc:0.97305


...

[2800]	train-auc:0.99385	valid-auc:0.98711


[2999]	train-auc:0.99426	valid-auc:0.98721


[2025-09-11 23:05:19] [XGB] Fold 9 AUC: 0.987210 | best_iter=2998 | elapsed=166.9s


[2025-09-11 23:05:20] [XGB] OOF AUC: 0.986782 | total_elapsed=1638.8s


[2025-09-11 23:05:22] [XGB] Saved OOF and test predictions.


In [13]:
# LightGBM multi-seed trainer (CPU-only) on current 165-feature set and locked folds
log('Preparing LightGBM multi-seed training cell (will run after XGB finishes)...')
import lightgbm as lgb

def train_lgb_fast_seed(seed=42, feature_fraction=0.75, bagging_fraction=0.75, tag=None):
    t0 = time.time()
    feats = [c for c in train.columns if c not in [id_col, target_col, 'f_27']]
    X_ = train[feats]; y_ = train[target_col].values; Xte_ = test[feats]
    folds_df = pd.read_csv('folds_10fold_seed42.csv'); folds_arr = folds_df['fold'].values
    n_splits = len(np.unique(folds_arr))
    params = {
        'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
        'learning_rate': 0.045, 'num_leaves': 144, 'max_depth': -1,
        'min_data_in_leaf': 240, 'feature_fraction': feature_fraction, 'bagging_fraction': bagging_fraction, 'bagging_freq': 1,
        'lambda_l1': 0.0, 'lambda_l2': 4.0, 'max_bin': 127, 'verbose': -1, 'n_jobs': -1,
        'seed': seed, 'feature_fraction_seed': seed, 'bagging_seed': seed, 'data_random_seed': seed
    }
    oof_ = np.zeros(len(train), dtype='float32'); pred_te_ = np.zeros(len(test), dtype='float32')
    for fold in range(n_splits):
        fold_t0 = time.time()
        trn_idx = np.where(folds_arr != fold)[0]; val_idx = np.where(folds_arr == fold)[0]
        log(f'[LGB multi] seed={seed} ff={feature_fraction} bf={bagging_fraction} | Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
        dtrain = lgb.Dataset(X_.iloc[trn_idx], label=y_[trn_idx])
        dvalid = lgb.Dataset(X_.iloc[val_idx], label=y_[val_idx])
        model = lgb.train(params, dtrain, num_boost_round=4000, valid_sets=[dtrain, dvalid], valid_names=['train','valid'], callbacks=[lgb.early_stopping(120), lgb.log_evaluation(200)])
        oof_[val_idx] = model.predict(X_.iloc[val_idx], num_iteration=model.best_iteration)
        auc = roc_auc_score(y_[val_idx], oof_[val_idx])
        log(f'[LGB multi] seed={seed} fold={fold} AUC: {auc:.6f} | best_iter={model.best_iteration} | elapsed={time.time()-fold_t0:.1f}s')
        pred_te_ += model.predict(Xte_, num_iteration=model.best_iteration) / n_splits
        del dtrain, dvalid, model; gc.collect()
    cv_auc = roc_auc_score(y_, oof_); log(f'[LGB multi] seed={seed} OOF AUC: {cv_auc:.6f} | total_elapsed={time.time()-t0:.1f}s')
    tag = tag if tag is not None else f'seed{seed}'
    pd.DataFrame({id_col: train[id_col], f'oof_lgb_fast_{tag}': oof_}).to_csv(f'oof_lgb_fast_{tag}.csv', index=False)
    pd.DataFrame({id_col: test[id_col], f'prediction_lgb_fast_{tag}': pred_te_}).to_csv(f'pred_lgb_fast_{tag}.csv', index=False)
    log(f'[LGB multi] Saved OOF/test preds for {tag}')

# Usage plan (execute after XGB finishes):
# train_lgb_fast_seed(42, 0.75, 0.75, tag='s42')
# train_lgb_fast_seed(1337, 0.72, 0.72, tag='s1337')
# train_lgb_fast_seed(2025, 0.78, 0.78, tag='s2025')

[2025-09-11 23:05:36] Preparing LightGBM multi-seed training cell (will run after XGB finishes)...


In [14]:
# Run LightGBM fast for seed 42 on current 165-feature set and locked folds
log('Starting LightGBM multi-seed training: seed 42...')
train_lgb_fast_seed(42, 0.75, 0.75, tag='s42')

[2025-09-11 23:06:01] Starting LightGBM multi-seed training: seed 42...


[2025-09-11 23:06:02] [LGB multi] seed=42 ff=0.75 bf=0.75 | Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 120 rounds


[200]	train's auc: 0.978775	valid's auc: 0.975702


[400]	train's auc: 0.98866	valid's auc: 0.985066


...

[3800]	train's auc: 0.999984	valid's auc: 0.991156


Early stopping, best iteration is:
[3732]	train's auc: 0.99998	valid's auc: 0.991168


[2025-09-11 23:58:19] [LGB multi] seed=42 fold=9 AUC: 0.991168 | best_iter=3732 | elapsed=303.4s


[2025-09-11 23:58:22] [LGB multi] seed=42 OOF AUC: 0.991021 | total_elapsed=3140.3s


[2025-09-11 23:58:23] [LGB multi] Saved OOF/test preds for s42


In [16]:
# Blending: LGB multi-seeds (s42,s1337,s2025) + XGB (seed42); rank/prob grid on OOF; apply to test
log('Preparing blend (LGB seeds + XGB) with OOF weight tuning...')
import glob

gt = train[['id', 'target']].copy()

def safe_load_oof(path, default_col):
    if not os.path.exists(path):
        return None
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    col = default_col if default_col in df.columns else (cols[0] if cols else None)
    return df[['id', col]].rename(columns={col: 'pred'}) if col else None

def safe_load_pred(path, default_col):
    if not os.path.exists(path):
        return None
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    col = default_col if default_col in df.columns else (cols[0] if cols else None)
    return df[['id', col]].rename(columns={col: 'pred'}) if col else None

# LGB seeds (new 165-feature runs)
oof_lgb_paths = [
    ('oof_lgb_fast_s42.csv', 'oof_lgb_fast_s42'),
    ('oof_lgb_fast_s1337.csv', 'oof_lgb_fast_s1337'),
    ('oof_lgb_fast_s2025.csv', 'oof_lgb_fast_s2025'),
]
pred_lgb_paths = [
    ('pred_lgb_fast_s42.csv', 'prediction_lgb_fast_s42'),
    ('pred_lgb_fast_s1337.csv', 'prediction_lgb_fast_s1337'),
    ('pred_lgb_fast_s2025.csv', 'prediction_lgb_fast_s2025'),
]

oof_lgb_list = []
for p, col in oof_lgb_paths:
    df = safe_load_oof(p, col)
    if df is not None:
        oof_lgb_list.append(df.rename(columns={'pred': f'pred_{p}'}))

pred_lgb_list = []
for p, col in pred_lgb_paths:
    df = safe_load_pred(p, col)
    if df is not None:
        pred_lgb_list.append(df.rename(columns={'pred': f'pred_{p}'}))

if len(oof_lgb_list) == 0:
    log('No LGB seed OOF files found. Run LGB seeds first.');
else:
    oof_lgb_merged = oof_lgb_list[0]
    for df in oof_lgb_list[1:]:
        oof_lgb_merged = oof_lgb_merged.merge(df, on='id', how='inner')
    # Build LGB ensemble OOF by simple average
    lgb_cols = [c for c in oof_lgb_merged.columns if c != 'id']
    oof_lgb_merged['lgb_ens'] = oof_lgb_merged[lgb_cols].mean(axis=1)
    oof = gt.merge(oof_lgb_merged[['id', 'lgb_ens']], on='id', how='left')

    # XGB (seed42)
    oof_xgb = safe_load_oof('oof_xgb_seed42.csv', 'oof_xgb')
    have_xgb = oof_xgb is not None
    if have_xgb:
        oof = oof.merge(oof_xgb.rename(columns={'pred': 'xgb'}), on='id', how='left')
    else:
        oof['xgb'] = np.nan

    # Rank normalization helper
    def rank_norm(x):
        r = pd.Series(x).rank(method='average').values
        return (r - 1) / (len(r) - 1)

    oof['lgb_r'] = rank_norm(oof['lgb_ens'])
    if have_xgb:
        oof['xgb_r'] = rank_norm(oof['xgb'])

    best_auc = 0.0; best_mode = 'prob'; best_w = 1.0
    weights = np.arange(0.50, 0.95, 0.05) if have_xgb else [1.0]
    for w in weights:
        if have_xgb:
            blend_prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
            blend_rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
        else:
            blend_prob = oof['lgb_ens']
            blend_rank = oof['lgb_r']
        auc_prob = roc_auc_score(oof['target'], blend_prob)
        if auc_prob > best_auc:
            best_auc = auc_prob; best_mode = 'prob'; best_w = float(w)
        auc_rank = roc_auc_score(oof['target'], blend_rank)
        if auc_rank > best_auc:
            best_auc = auc_rank; best_mode = 'rank'; best_w = float(w)

    log(f"Blend search best OOF AUC={best_auc:.6f} | mode={best_mode} | w_lgb={best_w:.2f}")

    # Build test ensemble
    if len(pred_lgb_list) == 0:
        log('No LGB seed test preds found. Cannot build submission.');
    else:
        pred_lgb_merged = pred_lgb_list[0]
        for df in pred_lgb_list[1:]:
            pred_lgb_merged = pred_lgb_merged.merge(df, on='id', how='inner')
        lgb_pred_cols = [c for c in pred_lgb_merged.columns if c != 'id']
        pred_lgb_merged['lgb_ens'] = pred_lgb_merged[lgb_pred_cols].mean(axis=1)

        pred_xgb = safe_load_pred('pred_xgb_seed42.csv', 'prediction_xgb') if have_xgb else None
        have_pred_xgb = pred_xgb is not None
        sub = pd.read_csv('sample_submission.csv')
        sub = sub[['id']].merge(pred_lgb_merged[['id','lgb_ens']], on='id', how='left')
        if best_mode == 'prob' and have_pred_xgb:
            sub = sub.merge(pred_xgb.rename(columns={'pred': 'xgb'}), on='id', how='left')
            sub['target'] = (best_w * sub['lgb_ens'] + (1 - best_w) * sub['xgb']).astype('float32')
        elif best_mode == 'rank' and have_pred_xgb:
            sub = sub.merge(pred_xgb.rename(columns={'pred': 'xgb'}), on='id', how='left')
            rl = pd.Series(sub['lgb_ens']).rank(method='average').values
            rx = pd.Series(sub['xgb']).rank(method='average').values
            rl = (rl - 1) / (len(rl) - 1); rx = (rx - 1) / (len(rx) - 1)
            sub['target'] = (best_w * rl + (1 - best_w) * rx).astype('float32')
        else:
            sub['target'] = sub['lgb_ens'].astype('float32')

        sub[['id','target']].to_csv('submission.csv', index=False)
        log('Saved blended submission.csv (or LGB-only if XGB missing).')

[2025-09-12 01:40:30] Preparing blend (LGB seeds + XGB) with OOF weight tuning...


[2025-09-12 01:40:36] Blend search best OOF AUC=0.991159 | mode=prob | w_lgb=0.90


[2025-09-12 01:40:36] Saved blended submission.csv (or LGB-only if XGB missing).


In [15]:
# Run LightGBM fast for seeds 1337 and 2025 sequentially on current 165-feature set
log('Starting LightGBM multi-seed training: seeds 1337 and 2025...')
train_lgb_fast_seed(1337, 0.72, 0.72, tag='s1337')
train_lgb_fast_seed(2025, 0.78, 0.78, tag='s2025')

[2025-09-11 23:58:35] Starting LightGBM multi-seed training: seeds 1337 and 2025...


[2025-09-11 23:58:36] [LGB multi] seed=1337 ff=0.72 bf=0.72 | Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 120 rounds


[200]	train's auc: 0.978403	valid's auc: 0.975425


[400]	train's auc: 0.988473	valid's auc: 0.984865


...

[3400]	train's auc: 0.999948	valid's auc: 0.991228


Early stopping, best iteration is:
[3317]	train's auc: 0.999935	valid's auc: 0.991232


[2025-09-12 01:40:08] [LGB multi] seed=2025 fold=9 AUC: 0.991232 | best_iter=3317 | elapsed=268.2s


[2025-09-12 01:40:10] [LGB multi] seed=2025 OOF AUC: 0.991116 | total_elapsed=3014.1s


[2025-09-12 01:40:11] [LGB multi] Saved OOF/test preds for s2025


In [17]:
# Archive additional submissions and blend metadata
log('Creating additional submission variants and archiving blend metadata...')
import json

def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / (len(r) - 1)

# Load OOF for recomputing weights/metrics
gt = train[['id','target']].copy()
oof_lgb_paths = [
    ('oof_lgb_fast_s42.csv', 'oof_lgb_fast_s42'),
    ('oof_lgb_fast_s1337.csv', 'oof_lgb_fast_s1337'),
    ('oof_lgb_fast_s2025.csv', 'oof_lgb_fast_s2025'),
]
oof_list = []
for p, col in oof_lgb_paths:
    if os.path.exists(p):
        df = pd.read_csv(p)
        pred_col = col if col in df.columns else [c for c in df.columns if c != 'id'][0]
        oof_list.append(df[['id', pred_col]].rename(columns={pred_col: os.path.splitext(os.path.basename(p))[0]}))

if len(oof_list) == 0:
    log('Missing LGB OOF files; cannot create variants.')
else:
    oof_merged = oof_list[0]
    for df in oof_list[1:]:
        oof_merged = oof_merged.merge(df, on='id', how='inner')
    lgb_cols = [c for c in oof_merged.columns if c != 'id']
    oof_merged['lgb_ens'] = oof_merged[lgb_cols].mean(axis=1).astype('float32')
    oof = gt.merge(oof_merged[['id','lgb_ens']], on='id', how='left')
    # XGB OOF
    have_xgb = os.path.exists('oof_xgb_seed42.csv')
    if have_xgb:
        oof_xgb = pd.read_csv('oof_xgb_seed42.csv')
        xcol = 'oof_xgb' if 'oof_xgb' in oof_xgb.columns else [c for c in oof_xgb.columns if c != 'id'][0]
        oof = oof.merge(oof_xgb[['id', xcol]].rename(columns={xcol: 'xgb'}), on='id', how='left')
    else:
        oof['xgb'] = np.nan
    # Rank cols
    oof['lgb_r'] = rank_norm(oof['lgb_ens'])
    if have_xgb:
        oof['xgb_r'] = rank_norm(oof['xgb'])

    # Grid search per guidance: [0.60..0.85] step 0.05 then refine ±0.04 step 0.02
    best = {'auc': 0.0, 'mode': 'prob', 'w_lgb': 1.0}
    if have_xgb:
        coarse = np.arange(0.60, 0.85 + 1e-9, 0.05)
        cand = set(np.clip(np.concatenate([coarse,
                                           np.unique(np.round((coarse[:,None] + np.array([-0.04,-0.02,0.02,0.04]))
                                                              .ravel(), 2))]), 0.0, 1.0))
        weights = sorted(list(cand))
    else:
        weights = [1.0]
    for w in weights:
        if have_xgb:
            prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
            rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
        else:
            prob = oof['lgb_ens']
            rank = oof['lgb_r']
        auc_p = roc_auc_score(oof['target'], prob)
        if auc_p > best['auc']:
            best.update({'auc': float(auc_p), 'mode': 'prob', 'w_lgb': float(w)})
        auc_r = roc_auc_score(oof['target'], rank)
        if auc_r > best['auc']:
            best.update({'auc': float(auc_r), 'mode': 'rank', 'w_lgb': float(w)})

    # Build test preds:
    # LGB 3-seed average
    pred_paths = [
        ('pred_lgb_fast_s42.csv', 'prediction_lgb_fast_s42'),
        ('pred_lgb_fast_s1337.csv', 'prediction_lgb_fast_s1337'),
        ('pred_lgb_fast_s2025.csv', 'prediction_lgb_fast_s2025'),
    ]
    preds = []
    for p, col in pred_paths:
        if os.path.exists(p):
            df = pd.read_csv(p)
            pcol = col if col in df.columns else [c for c in df.columns if c != 'id'][0]
            preds.append(df[['id', pcol]].rename(columns={pcol: os.path.splitext(os.path.basename(p))[0]}))
    if len(preds) < 3:
        log('Missing some LGB test predictions; skipping extra submissions.')
    else:
        pm = preds[0]
        for df in preds[1:]:
            pm = pm.merge(df, on='id', how='inner')
        lgb_pred_cols = [c for c in pm.columns if c != 'id']
        pm['lgb_ens'] = pm[lgb_pred_cols].mean(axis=1).astype('float32')

        # Save pure 3-seed LGB average submission
        ss = pd.read_csv('sample_submission.csv')[['id']].merge(pm[['id','lgb_ens']], on='id', how='left')
        ss = ss.rename(columns={'lgb_ens': 'target'})
        ss.to_csv('submission_lgb3.csv', index=False)
        log('Saved submission_lgb3.csv (pure LGB 3-seed average).')

        # Fixed rank blend 0.72/0.28 with XGB
        if have_xgb and os.path.exists('pred_xgb_seed42.csv'):
            pred_xgb = pd.read_csv('pred_xgb_seed42.csv')
            xcol = 'prediction_xgb' if 'prediction_xgb' in pred_xgb.columns else [c for c in pred_xgb.columns if c != 'id'][0]
            sub_fx = pd.read_csv('sample_submission.csv')[['id']].merge(pm[['id','lgb_ens']], on='id', how='left')
            sub_fx = sub_fx.merge(pred_xgb[['id', xcol]].rename(columns={xcol: 'xgb'}), on='id', how='left')
            rl = pd.Series(sub_fx['lgb_ens']).rank(method='average').values
            rx = pd.Series(sub_fx['xgb']).rank(method='average').values
            rl = (rl - 1) / (len(rl) - 1); rx = (rx - 1) / (len(rx) - 1)
            sub_fx['target'] = (0.72 * rl + 0.28 * rx).astype('float32')
            sub_fx[['id','target']].to_csv('submission_rank_072_028.csv', index=False)
            log('Saved submission_rank_072_028.csv (fixed rank 0.72/0.28).')
        else:
            log('XGB preds missing; skipped fixed rank 0.72/0.28 file.')

        # Archive current best submission.csv as submission_best.csv if present
        if os.path.exists('submission.csv'):
            sub_best = pd.read_csv('submission.csv')
            sub_best.to_csv('submission_best.csv', index=False)
            log('Archived current submission.csv as submission_best.csv')

    # Save blend metadata
    meta = {
        'best_oof_auc': best['auc'],
        'best_mode': best['mode'],
        'best_w_lgb': best['w_lgb'],
        'notes': 'Weights searched on OOF with LGB 3-seed ensemble vs XGB; additional variants saved.'
    }
    with open('blend_meta.json', 'w') as f:
        json.dump(meta, f, indent=2)
    log(f"Saved blend_meta.json: {meta}")

[2025-09-12 01:41:58] Creating additional submission variants and archiving blend metadata...


[2025-09-12 01:42:16] Saved submission_lgb3.csv (pure LGB 3-seed average).


[2025-09-12 01:42:16] Saved submission_rank_072_028.csv (fixed rank 0.72/0.28).


[2025-09-12 01:42:16] Archived current submission.csv as submission_best.csv


[2025-09-12 01:42:16] Saved blend_meta.json: {'best_oof_auc': 0.991134746878686, 'best_mode': 'prob', 'best_w_lgb': 0.89, 'notes': 'Weights searched on OOF with LGB 3-seed ensemble vs XGB; additional variants saved.'}


In [19]:
# Strictly rebuild current blended submission to ensure format compliance
log('Strictly rebuilding submission.csv from current blended predictions...')
assert os.path.exists('submission.csv'), 'submission.csv not found to rebuild'
ss = pd.read_csv('sample_submission.csv')[['id']]
sub_cur = pd.read_csv('submission.csv')
# Detect prediction column
pred_col = None
if list(sub_cur.columns) == ['id', 'target']:
    pred_col = 'target'
else:
    # find non-id column
    other = [c for c in sub_cur.columns if c != 'id']
    pred_col = other[0] if other else None
assert pred_col is not None, 'No prediction column found in submission.csv'
# Merge to enforce ID order and exact length
rebuilt = ss.merge(sub_cur[['id', pred_col]].rename(columns={pred_col: 'target'}), on='id', how='left')
missing = int(rebuilt['target'].isna().sum())
if missing > 0:
    raise ValueError(f'Missing predictions for {missing} test rows after strict rebuild')
# Enforce numeric type and [0,1] bounds
rebuilt['target'] = pd.to_numeric(rebuilt['target'], errors='coerce')
if rebuilt['target'].isna().any():
    raise ValueError('NaNs present after numeric coercion in target')
rebuilt['target'] = rebuilt['target'].astype('float64').clip(0.0, 1.0)
rebuilt = rebuilt[['id', 'target']]
rebuilt.to_csv('submission.csv', index=False)
log('submission.csv strictly rebuilt and saved (float64, clipped [0,1], aligned to sample_submission).')

[2025-09-12 01:43:11] Strictly rebuilding submission.csv from current blended predictions...


[2025-09-12 01:43:11] submission.csv strictly rebuilt and saved (float64, clipped [0,1], aligned to sample_submission).


In [21]:
# Pivot pipeline: f_27 identity map and compact numeric-only features (no f_27-derived) for unseen subset
log('Pivot: building f_27 identity map for seen rows and compact numeric-only features for unseen model...')
t0 = time.time()

# 1) f_27 -> target map from train; identify seen/unseen in test
assert 'f_27' in train.columns and 'target' in train.columns, 'Missing f_27 or target in train'
f27_target_map = train.groupby('f_27')['target'].first().to_dict()
test_seen_mask = test['f_27'].isin(f27_target_map)
n_seen = int(test_seen_mask.sum())
n_unseen = int((~test_seen_mask).sum())
log(f'Test seen f_27 rows: {n_seen} | unseen: {n_unseen}')
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_target_map).astype('float32')

# 2) Define compact numeric-only feature set
base_num_cols = [c for c in train.columns if c.startswith('f_') and c != 'f_27' and (str(train[c].dtype).startswith('float') or str(train[c].dtype).startswith('int'))]
# Keep only original numeric features f_00..f_26 (not any previously engineered columns)
base_num_cols = [c for c in base_num_cols if len(c) == 4 and c[2:].isdigit()]
base_num_cols = sorted(base_num_cols)
log(f'Base numeric cols count: {len(base_num_cols)}')

# Build compact extra features: sorted row stats, equality count over selected numerics, parity bits
sel_eq_cols = [c for c in ['f_00','f_01','f_02','f_03','f_05','f_06','f_10','f_12','f_20','f_21','f_22','f_26'] if c in base_num_cols]
parity_cols = [c for c in ['f_00','f_02','f_10','f_20','f_21','f_26'] if c in base_num_cols]

def build_compact_block(df):
    vals = df[base_num_cols].astype('float32').values
    srt = np.sort(vals, axis=1)
    out = pd.DataFrame(index=df.index)
    out['comp_sorted_0'] = srt[:, 0].astype('float32')
    out['comp_sorted_1'] = srt[:, 1].astype('float32')
    out['comp_sorted_-1'] = srt[:, -1].astype('float32')
    out['comp_sorted_range'] = (srt[:, -1] - srt[:, 0]).astype('float32')
    # equality count over selected numerics
    if len(sel_eq_cols) >= 2:
        M = df[sel_eq_cols].astype('float32').values
        from itertools import combinations
        cnt = np.zeros(len(df), dtype=np.int16)
        for i, j in combinations(range(M.shape[1]), 2):
            cnt += (np.abs(M[:, i] - M[:, j]) < 1e-6)
        out['comp_num_equal_pairs'] = cnt
    else:
        out['comp_num_equal_pairs'] = 0
    # parity bits
    for c in parity_cols:
        out[f'comp_{c}_parity'] = (df[c].astype('int64') & 1).astype('int8')
    return out

comp_train = build_compact_block(train)
comp_test = build_compact_block(test)

# Track compact feature names explicitly
compact_feature_cols = list(comp_train.columns)
log(f'Compact feature count: {len(compact_feature_cols)}')

# 3) Persist helper artifacts in memory (globals) and basic masks for later cells
test_unseen_mask = ~test_seen_mask
test_unseen_f27 = set(test.loc[test_unseen_mask, 'f_27'].unique())
train_mask_unseen_vocab = train['f_27'].isin(test_unseen_f27).values  # for OOF weighting on unseen-like rows
log(f'Train rows in unseen test vocab: {int(train_mask_unseen_vocab.sum())} / {len(train)}')

# 4) Sanity: no NaNs, and ensure we do not include any f_27-derived feature
assert all(['f_27' not in c for c in compact_feature_cols]), 'Compact features contaminated with f_27-derived cols'
nan_tr = int(comp_train.isna().sum().sum()); nan_te = int(comp_test.isna().sum().sum())
assert nan_tr == 0 and nan_te == 0, f'NaNs in compact features: train {nan_tr}, test {nan_te}'

log(f'Pivot prep done in {time.time()-t0:.1f}s. Ready to train unseen-subset models.')

[2025-09-12 01:47:08] Pivot: building f_27 identity map for seen rows and compact numeric-only features for unseen model...


[2025-09-12 01:47:10] Test seen f_27 rows: 28435 | unseen: 71565


[2025-09-12 01:47:10] Base numeric cols count: 30


[2025-09-12 01:47:11] Compact feature count: 11


[2025-09-12 01:47:11] Train rows in unseen test vocab: 0 / 800000


[2025-09-12 01:47:11] Pivot prep done in 3.3s. Ready to train unseen-subset models.


In [22]:
# Train compact-feature LightGBM models (5 seeds) for unseen subset
log('Training compact-feature LightGBM models for unseen subset...')
import lightgbm as lgb

assert 'comp_sorted_0' in comp_train.columns, 'Compact features not built; run pivot prep cell first.'
Xc = comp_train[compact_feature_cols].copy()
yc = train['target'].values
Xc_test = comp_test[compact_feature_cols].copy()

# Reuse locked folds
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

def train_lgb_compact(seed=42, ff=0.75, bf=0.75, tag='c42'):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': 0.045,
        'num_leaves': 160,
        'max_depth': -1,
        'min_data_in_leaf': 240,
        'feature_fraction': ff,
        'bagging_fraction': bf,
        'bagging_freq': 1,
        'lambda_l1': 0.0,
        'lambda_l2': 4.0,
        'max_bin': 127,
        'verbose': -1,
        'n_jobs': -1,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'data_random_seed': seed
    }
    oof = np.zeros(len(Xc), dtype='float32')
    ptest = np.zeros(len(Xc_test), dtype='float32')
    t0 = time.time()
    for fold in range(n_splits):
        trn_idx = np.where(folds_arr != fold)[0]
        val_idx = np.where(folds_arr == fold)[0]
        log(f'[LGB compact] seed={seed} ff={ff} bf={bf} | Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
        dtr = lgb.Dataset(Xc.iloc[trn_idx], label=yc[trn_idx])
        dvl = lgb.Dataset(Xc.iloc[val_idx], label=yc[val_idx])
        model = lgb.train(
            params,
            dtr,
            num_boost_round=4000,
            valid_sets=[dtr, dvl],
            valid_names=['train','valid'],
            callbacks=[lgb.early_stopping(120), lgb.log_evaluation(200)]
        )
        oof[val_idx] = model.predict(Xc.iloc[val_idx], num_iteration=model.best_iteration)
        auc = roc_auc_score(yc[val_idx], oof[val_idx])
        log(f'[LGB compact] seed={seed} fold={fold} AUC: {auc:.6f} | best_iter={model.best_iteration}')
        ptest += model.predict(Xc_test, num_iteration=model.best_iteration) / n_splits
        del dtr, dvl, model; gc.collect()
    cv_auc = roc_auc_score(yc, oof);
    log(f'[LGB compact] seed={seed} OOF AUC: {cv_auc:.6f} | elapsed={time.time()-t0:.1f}s')
    pd.DataFrame({'id': train['id'], f'oof_lgb_comp_{tag}': oof}).to_csv(f'oof_lgb_comp_{tag}.csv', index=False)
    pd.DataFrame({'id': test['id'], f'prediction_lgb_comp_{tag}': ptest}).to_csv(f'pred_lgb_comp_{tag}.csv', index=False)
    log(f'[LGB compact] Saved OOF/test preds for {tag}')

# Run 5 seeds sequentially
seed_cfgs = [
    (42, 0.75, 0.75, 'c42'),
    (1337, 0.72, 0.72, 'c1337'),
    (2025, 0.78, 0.78, 'c2025'),
    (101, 0.70, 0.80, 'c101'),
    (999, 0.80, 0.70, 'c999'),
]
for s, ff, bf, tag in seed_cfgs:
    train_lgb_compact(s, ff, bf, tag)
log('All compact LGB seeds finished.')

[2025-09-12 01:48:00] Training compact-feature LightGBM models for unseen subset...


[2025-09-12 01:48:00] [LGB compact] seed=42 ff=0.75 bf=0.75 | Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 120 rounds


[200]	train's auc: 0.609486	valid's auc: 0.577636


Early stopping, best iteration is:
[131]	train's auc: 0.601722	valid's auc: 0.578104
[2025-09-12 01:48:13] [LGB compact] seed=42 fold=0 AUC: 0.578104 | best_iter=131


...

[200]	train's auc: 0.609776	valid's auc: 0.579659


Early stopping, best iteration is:
[164]	train's auc: 0.605749	valid's auc: 0.579906
[2025-09-12 01:58:34] [LGB compact] seed=999 fold=9 AUC: 0.579906 | best_iter=164


[2025-09-12 01:58:34] [LGB compact] seed=999 OOF AUC: 0.580228 | elapsed=120.3s


[2025-09-12 01:58:35] [LGB compact] Saved OOF/test preds for c999


[2025-09-12 01:58:35] All compact LGB seeds finished.


In [23]:
# Compact-feature XGBoost (CPU, hist) training for unseen subset diversity
log('Training compact-feature XGBoost (CPU, hist) for unseen subset...')
t0_all = time.time()
try:
    import xgboost as xgb
except ImportError:
    import sys, subprocess
    log('XGBoost not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb

assert 'comp_sorted_0' in comp_train.columns, 'Compact features not built; run pivot prep first.'
Xc = comp_train[compact_feature_cols].copy()
yc = train['target'].values
Xc_test = comp_test[compact_feature_cols].copy()

# Folds (locked)
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

# Params (compact set, slightly deeper for diversity)
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'max_bin': 128,
    'eta': 0.045,
    'max_depth': 8,
    'min_child_weight': 80,
    'subsample': 0.78,
    'colsample_bytree': 0.9,
    'lambda': 3.0,
    'nthread': 36,
    'seed': 42
}
num_boost_round = 3000
early_stopping_rounds = 150

oof_xgb_c = np.zeros(len(Xc), dtype='float32')
pred_test_xgb_c = np.zeros(len(Xc_test), dtype='float32')

for fold in range(n_splits):
    fold_t0 = time.time()
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[XGB compact] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
    dtrain = xgb.DMatrix(Xc.iloc[trn_idx], label=yc[trn_idx])
    dvalid = xgb.DMatrix(Xc.iloc[val_idx], label=yc[val_idx])
    dtest = xgb.DMatrix(Xc_test)
    bst = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=200
    )
    best_iter = None
    try:
        attrs = bst.attributes()
        if 'best_iteration' in attrs:
            best_iter = int(attrs['best_iteration'])
    except Exception:
        best_iter = getattr(bst, 'best_iteration', None)
    if best_iter is not None and best_iter >= 0:
        oof_pred = bst.predict(dvalid, iteration_range=(0, best_iter + 1))
    else:
        oof_pred = bst.predict(dvalid)
    oof_xgb_c[val_idx] = oof_pred.astype('float32')
    fold_auc = roc_auc_score(yc[val_idx], oof_xgb_c[val_idx])
    log(f'[XGB compact] Fold {fold} AUC: {fold_auc:.6f} | best_iter={best_iter} | elapsed={time.time()-fold_t0:.1f}s')
    if best_iter is not None and best_iter >= 0:
        pred_te = bst.predict(dtest, iteration_range=(0, best_iter + 1))
    else:
        pred_te = bst.predict(dtest)
    pred_test_xgb_c += pred_te.astype('float32') / n_splits
    del dtrain, dvalid, dtest, bst
    gc.collect()

cv_auc_xgb_c = roc_auc_score(yc, oof_xgb_c)
log(f'[XGB compact] OOF AUC: {cv_auc_xgb_c:.6f} | total_elapsed={time.time()-t0_all:.1f}s')
pd.DataFrame({'id': train['id'], 'oof_xgb_comp': oof_xgb_c}).to_csv('oof_xgb_comp.csv', index=False)
pd.DataFrame({'id': test['id'], 'prediction_xgb_comp': pred_test_xgb_c}).to_csv('pred_xgb_comp.csv', index=False)
log('[XGB compact] Saved OOF and test predictions.')

[2025-09-12 01:58:44] Training compact-feature XGBoost (CPU, hist) for unseen subset...


[2025-09-12 01:58:45] [XGB compact] Fold 1/10 | trn=720000 val=80000


[0]	train-auc:0.57450	valid-auc:0.56864


[200]	train-auc:0.59887	valid-auc:0.57803


[400]	train-auc:0.60925	valid-auc:0.57773


...

[400]	train-auc:0.60881	valid-auc:0.57882


[444]	train-auc:0.61080	valid-auc:0.57854


[2025-09-12 02:00:34] [XGB compact] Fold 9 AUC: 0.579440 | best_iter=294 | elapsed=15.2s


[2025-09-12 02:00:35] [XGB compact] OOF AUC: 0.580264 | total_elapsed=110.3s


[2025-09-12 02:00:36] [XGB compact] Saved OOF and test predictions.


In [24]:
# Assemble final submission using f_27 identity for seen and compact-model ensemble for unseen
log('Assembling final submission: f_27 identity for seen; compact-model ensemble for unseen...')
import json

# 1) Recompute f_27 -> target map and seen/unseen masks
f27_target_map = train.groupby('f_27')['target'].first().to_dict()
test_seen_mask = test['f_27'].isin(f27_target_map).values
test_unseen_mask = ~test_seen_mask
log(f'Seen test rows: {int(test_seen_mask.sum())} | Unseen: {int(test_unseen_mask.sum())}')
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_target_map).astype('float32').values

# 2) Load compact LGB OOF/test preds and form ensemble
def load_comp_oof(path):
    if not os.path.exists(path):
        return None
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    if len(cols) != 1:
        return None
    return df.rename(columns={cols[0]: 'pred'})

def load_comp_pred(path):
    if not os.path.exists(path):
        return None
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    if len(cols) != 1:
        return None
    return df.rename(columns={cols[0]: 'pred'})

lgb_tags = ['c42','c1337','c2025','c101','c999']
oofs = []; preds = []
for tag in lgb_tags:
    po = load_comp_oof(f'oof_lgb_comp_{tag}.csv')
    pt = load_comp_pred(f'pred_lgb_comp_{tag}.csv')
    if (po is not None) and (pt is not None):
        oofs.append(po.rename(columns={'pred': f'oof_{tag}'}))
        preds.append(pt.rename(columns={'pred': f'pred_{tag}'}))

if len(oofs) == 0 or len(preds) == 0:
    raise RuntimeError('Missing compact LGB artifacts; train compact models first.')

oof_lgb = oofs[0]
for df in oofs[1:]:
    oof_lgb = oof_lgb.merge(df, on='id', how='inner')
pred_lgb = preds[0]
for df in preds[1:]:
    pred_lgb = pred_lgb.merge(df, on='id', how='inner')
lgb_oof_cols = [c for c in oof_lgb.columns if c != 'id']
lgb_pred_cols = [c for c in pred_lgb.columns if c != 'id']
oof_lgb['lgb_ens'] = oof_lgb[lgb_oof_cols].mean(axis=1).astype('float32')
pred_lgb['lgb_ens'] = pred_lgb[lgb_pred_cols].mean(axis=1).astype('float32')

# 3) Load compact XGB OOF/test if available
oof_xgb = load_comp_oof('oof_xgb_comp.csv')
pred_xgb = load_comp_pred('pred_xgb_comp.csv')
have_xgb = (oof_xgb is not None) and (pred_xgb is not None)

# 4) Tune LGB/XGB weight on OOF (global fallback if no unseen-like train rows)
gt = train[['id','target']].copy()
oof = gt.merge(oof_lgb[['id','lgb_ens']], on='id', how='left')
if have_xgb:
    oof = oof.merge(oof_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')
else:
    oof['xgb'] = np.nan

def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / (len(r) - 1)

oof['lgb_r'] = rank_norm(oof['lgb_ens'])
if have_xgb:
    oof['xgb_r'] = rank_norm(oof['xgb'])

best = {'auc': 0.0, 'mode': 'prob', 'w_lgb': 1.0}
if have_xgb:
    coarse = np.arange(0.70, 0.85 + 1e-9, 0.05)
    refine = np.unique(np.round((coarse[:,None] + np.array([-0.04,-0.02,0.02,0.04])).ravel(), 2))
    weights = sorted(list(set(np.clip(np.concatenate([coarse, refine]), 0.0, 1.0))))
else:
    weights = [1.0]
for w in weights:
    if have_xgb:
        prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
        rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
    else:
        prob = oof['lgb_ens']
        rank = oof['lgb_r']
    auc_p = roc_auc_score(oof['target'], prob)
    if auc_p > best['auc']:
        best.update({'auc': float(auc_p), 'mode': 'prob', 'w_lgb': float(w)})
    auc_r = roc_auc_score(oof['target'], rank)
    if auc_r > best['auc']:
        best.update({'auc': float(auc_r), 'mode': 'rank', 'w_lgb': float(w)})
log(f"[Unseen blend] Best OOF AUC={best['auc']:.6f} | mode={best['mode']} | w_lgb={best['w_lgb']}")

# 5) Build unseen test predictions
sub = pd.read_csv('sample_submission.csv')[['id']]
sub = sub.merge(pred_lgb[['id','lgb_ens']], on='id', how='left')
if have_xgb:
    sub = sub.merge(pred_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')
if best['mode'] == 'prob' or not have_xgb:
    unseen_pred = sub['lgb_ens'].values if not have_xgb else (best['w_lgb'] * sub['lgb_ens'].values + (1 - best['w_lgb']) * sub['xgb'].values)
else:
    rl = pd.Series(sub['lgb_ens']).rank(method='average').values
    rx = pd.Series(sub['xgb']).rank(method='average').values
    rl = (rl - 1) / (len(rl) - 1); rx = (rx - 1) / (len(rx) - 1)
    unseen_pred = (best['w_lgb'] * rl + (1 - best['w_lgb']) * rx)
unseen_pred = unseen_pred.astype('float32')

# 6) Assemble final submission: seen via map, unseen via blended preds
final = pd.read_csv('sample_submission.csv')[['id']].copy()
final['target'] = np.zeros(len(final), dtype='float32')
final.loc[test_seen_mask, 'target'] = preds_seen
final.loc[test_unseen_mask, 'target'] = unseen_pred[test_unseen_mask]

# 7) Sanity checks and save
if final['target'].isna().any():
    raise ValueError('NaNs found in final target after assembly')
final['target'] = final['target'].astype('float64').clip(0.0, 1.0)
final.to_csv('submission.csv', index=False)
log('Saved submission.csv (seen from f_27 map, unseen from blended compact ensemble).')

# Extra variants: pure LGB unseen and fixed 0.75/0.25 prob blend for unseen
final_lgb = final.copy()
final_lgb.loc[test_unseen_mask, 'target'] = sub.loc[test_unseen_mask, 'lgb_ens'].astype('float32')
final_lgb['target'] = final_lgb['target'].astype('float64').clip(0.0,1.0)
final_lgb.to_csv('submission_unseen_lgb.csv', index=False)
log('Saved submission_unseen_lgb.csv (unseen from LGB-avg only).')
if have_xgb:
    final_fx = final.copy()
    fx = (0.75 * sub['lgb_ens'].values + 0.25 * (sub['xgb'].values if have_xgb else 0.0)).astype('float32')
    final_fx.loc[test_unseen_mask, 'target'] = fx[test_unseen_mask]
    final_fx['target'] = final_fx['target'].astype('float64').clip(0.0,1.0)
    final_fx.to_csv('submission_unseen_prob_075_025.csv', index=False)
    log('Saved submission_unseen_prob_075_025.csv (unseen prob 0.75/0.25).')

# Save meta
with open('blend_unseen_meta.json','w') as f:
    json.dump(best, f, indent=2)
log(f"Saved blend_unseen_meta.json: {best}")

[2025-09-12 02:00:51] Assembling final submission: f_27 identity for seen; compact-model ensemble for unseen...


[2025-09-12 02:00:53] Seen test rows: 28435 | Unseen: 71565


[2025-09-12 02:01:07] [Unseen blend] Best OOF AUC=0.581052 | mode=prob | w_lgb=0.66


[2025-09-12 02:01:07] Saved submission.csv (seen from f_27 map, unseen from blended compact ensemble).


[2025-09-12 02:01:07] Saved submission_unseen_lgb.csv (unseen from LGB-avg only).


[2025-09-12 02:01:07] Saved submission_unseen_prob_075_025.csv (unseen prob 0.75/0.25).


[2025-09-12 02:01:07] Saved blend_unseen_meta.json: {'auc': 0.581052351856858, 'mode': 'prob', 'w_lgb': 0.66}


In [25]:
# Hybrid assembly: f_27 identity for seen; best LGB(3-seed)+XGB blend for unseen (full 165-feature models)
log('Building hybrid submission: identity map for seen, strong full-feature blend for unseen...')

# 1) f_27 -> target identity map and masks
f27_target_map = train.groupby('f_27')['target'].first().to_dict()
t_seen_mask = test['f_27'].isin(f27_target_map).values
t_unseen_mask = ~t_seen_mask
log(f'Hybrid masks -> seen: {int(t_seen_mask.sum())} | unseen: {int(t_unseen_mask.sum())}')
preds_seen = test.loc[t_seen_mask, 'f_27'].map(f27_target_map).astype('float32').values

# 2) Load OOF/test for 3-seed LGB (full features) and XGB, create tuned blend
def _load_oof(path):
    if not os.path.exists(path): return None
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})
def _load_pred(path):
    if not os.path.exists(path): return None
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})

# LGB seeds OOF
oofs = []; preds = [];
for p_o, p_t in [
    ('oof_lgb_fast_s42.csv','pred_lgb_fast_s42.csv'),
    ('oof_lgb_fast_s1337.csv','pred_lgb_fast_s1337.csv'),
    ('oof_lgb_fast_s2025.csv','pred_lgb_fast_s2025.csv'),
]:
    oo = _load_oof(p_o); pt = _load_pred(p_t)
    if (oo is not None) and (pt is not None):
        oofs.append(oo); preds.append(pt)
assert len(oofs) >= 1 and len(preds) >= 1, 'Missing full LGB seed artifacts'
oof_lgb = oofs[0]
for df in oofs[1:]: oof_lgb = oof_lgb.merge(df, on='id', how='inner', suffixes=('','_x'))
pred_lgb = preds[0]
for df in preds[1:]: pred_lgb = pred_lgb.merge(df, on='id', how='inner', suffixes=('','_x'))
lgb_oof_cols = [c for c in oof_lgb.columns if c != 'id']
lgb_pred_cols = [c for c in pred_lgb.columns if c != 'id']
oof_lgb['lgb_ens'] = oof_lgb[lgb_oof_cols].mean(axis=1).astype('float32')
pred_lgb['lgb_ens'] = pred_lgb[lgb_pred_cols].mean(axis=1).astype('float32')

# XGB OOF/pred
oof_xgb = _load_oof('oof_xgb_seed42.csv')
pred_xgb = _load_pred('pred_xgb_seed42.csv')
have_xgb = (oof_xgb is not None) and (pred_xgb is not None)

# 3) Tune blend weight on OOF (rank vs prob), using full-train OOF since unseen-like train rows are none
gt = train[['id','target']].copy()
oof = gt.merge(oof_lgb[['id','lgb_ens']], on='id', how='left')
if have_xgb: oof = oof.merge(oof_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')
else: oof['xgb'] = np.nan
def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / (len(r) - 1)
oof['lgb_r'] = rank_norm(oof['lgb_ens'])
if have_xgb: oof['xgb_r'] = rank_norm(oof['xgb'])
best = {'auc':0.0, 'mode':'prob', 'w':1.0}
weights = np.arange(0.60, 0.86, 0.05) if have_xgb else [1.0]
for w in weights:
    if have_xgb:
        prob = w*oof['lgb_ens'] + (1-w)*oof['xgb']
        rank = w*oof['lgb_r'] + (1-w)*oof['xgb_r']
    else:
        prob = oof['lgb_ens']; rank = oof['lgb_r']
    ap = roc_auc_score(oof['target'], prob)
    if ap > best['auc']: best.update({'auc':float(ap), 'mode':'prob', 'w':float(w)})
    ar = roc_auc_score(oof['target'], rank)
    if ar > best['auc']: best.update({'auc':float(ar), 'mode':'rank', 'w':float(w)})
log(f"[Hybrid blend] Best OOF AUC={best['auc']:.6f} | mode={best['mode']} | w_lgb={best['w']}")

# 4) Build unseen predictions from best blend
sub_te = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left')
if have_xgb: sub_te = sub_te.merge(pred_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')
if best['mode']=='prob' or not have_xgb:
    unseen_pred_full = sub_te['lgb_ens'].values if not have_xgb else (best['w']*sub_te['lgb_ens'].values + (1-best['w'])*sub_te['xgb'].values)
else:
    rl = rank_norm(sub_te['lgb_ens']); rx = rank_norm(sub_te['xgb'])
    unseen_pred_full = (best['w']*rl + (1-best['w'])*rx)
unseen_pred_full = unseen_pred_full.astype('float32')

# 5) Assemble final: seen from identity map; unseen from full-feature blend
final = pd.read_csv('sample_submission.csv')[['id']].copy()
final['target'] = np.zeros(len(final), dtype='float32')
final.loc[t_seen_mask, 'target'] = preds_seen
final.loc[t_unseen_mask, 'target'] = unseen_pred_full[t_unseen_mask]
final['target'] = final['target'].astype('float64').clip(0.0,1.0)
final.to_csv('submission.csv', index=False)
log('Hybrid submission.csv saved (seen: identity map; unseen: full-feature LGB+XGB blend).')

[2025-09-12 02:02:40] Building hybrid submission: identity map for seen, strong full-feature blend for unseen...


[2025-09-12 02:02:42] Hybrid masks -> seen: 28435 | unseen: 71565


[2025-09-12 02:02:47] [Hybrid blend] Best OOF AUC=0.991016 | mode=prob | w_lgb=0.8500000000000002


[2025-09-12 02:02:47] Hybrid submission.csv saved (seen: identity map; unseen: full-feature LGB+XGB blend).


In [27]:
# Leakage-safe Target Encodings (TE) for f_27 components + target-free frequencies (for unseen model)
log('Building OOF target encodings for f_27 positional chars and bigrams, plus frequency encodings...')
t0 = time.time()

assert 'f_27' in train.columns and 'target' in train.columns, 'Missing f_27/target'
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values.astype('int16')
n_splits = int(np.unique(folds_arr).size)
prior = float(train['target'].mean())

# Precompute tokens
s_tr = train['f_27'].astype(str).values
s_te = test['f_27'].astype(str).values

# Positional char integer tokens (already in columns f_27_pos_i but recompute arrays to be safe)
pos_char_tr = np.stack([train[f'f_27_pos_{i}'].astype('int16').values for i in range(10)], axis=1)
pos_char_te = np.stack([test[f'f_27_pos_{i}'].astype('int16').values for i in range(10)], axis=1)

# Positional bigram tokens (string form for groupby; train/test)
def make_bigrams(arr):
    # arr: array of strings length 10
    n = arr.shape[0]
    out = [None]*9
    for i in range(9):
        out[i] = np.fromiter((row[i:i+2] for row in arr), count=n, dtype=object)
    return out
bg_tr = make_bigrams(s_tr)
bg_te = make_bigrams(s_te)

# Optional: f27_nunique per row
def f27_nunique(arr):
    return np.fromiter((len(set(list(x))) for x in arr), count=len(arr), dtype=np.int16)
f27_nuniq_tr = f27_nunique(s_tr)
f27_nuniq_te = f27_nunique(s_te)

# Helper to compute OOF TE given key arrays (keys) and smoothing m
def oof_te_by_key(keys, y, m, prior):
    # keys: array-like of hashable tokens for all rows in train
    keys = pd.Series(keys)
    oof_vals = np.zeros(len(keys), dtype=np.float32)
    oof_logcnt = np.zeros(len(keys), dtype=np.float32)
    for fold in range(n_splits):
        trn_idx = np.where(folds_arr != fold)[0]
        val_idx = np.where(folds_arr == fold)[0]
        k_tr = keys.iloc[trn_idx]
        y_tr = y[trn_idx]
        grp = pd.DataFrame({'k': k_tr.values, 'y': y_tr}).groupby('k')
        cnt = grp['y'].size()
        sumy = grp['y'].sum()
        te_map = ((sumy + m * prior) / (cnt + m)).astype('float32')
        map_cnt = cnt.astype('int32')
        # map for val
        kval = keys.iloc[val_idx]
        te = kval.map(te_map).astype('float32')
        lc = kval.map(map_cnt).fillna(0).astype('int32')
        te = te.fillna(prior).values
        oof_vals[val_idx] = te
        oof_logcnt[val_idx] = np.log1p(lc.values).astype('float32')
    # Full fit for test transform
    grp_full = pd.DataFrame({'k': keys.values, 'y': y}).groupby('k')
    cnt_full = grp_full['y'].size()
    sumy_full = grp_full['y'].sum()
    te_map_full = ((sumy_full + m * prior) / (cnt_full + m)).astype('float32')
    cnt_full = cnt_full.astype('int32')
    return oof_vals, oof_logcnt, te_map_full, cnt_full

# Build TE for positional chars (10) with m=30
y_tr = train['target'].values.astype('float32')
te_pos_char_tr = np.zeros((len(train), 10), dtype=np.float32)
te_pos_char_logcnt_tr = np.zeros((len(train), 10), dtype=np.float32)
te_pos_char_te = np.zeros((len(test), 10), dtype=np.float32)
te_pos_char_logcnt_te = np.zeros((len(test), 10), dtype=np.float32)
for i in range(10):
    keys_tr = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[:, i]])
    oof_mean, oof_logcnt, te_map_full, cnt_full = oof_te_by_key(keys_tr, y_tr, m=30.0, prior=prior)
    te_pos_char_tr[:, i] = oof_mean
    te_pos_char_logcnt_tr[:, i] = oof_logcnt
    keys_te = pd.Series([f'{i}|{int(t)}' for t in pos_char_te[:, i]])
    te_vals = keys_te.map(te_map_full).astype('float32').fillna(prior).values
    lc = keys_te.map(cnt_full).fillna(0).astype('int32').values
    te_pos_char_te[:, i] = te_vals
    te_pos_char_logcnt_te[:, i] = np.log1p(lc).astype('float32')

# Build TE for positional bigrams (9) with m=100
te_pos_bg_tr = np.zeros((len(train), 9), dtype=np.float32)
te_pos_bg_logcnt_tr = np.zeros((len(train), 9), dtype=np.float32)
te_pos_bg_te = np.zeros((len(test), 9), dtype=np.float32)
te_pos_bg_logcnt_te = np.zeros((len(test), 9), dtype=np.float32)
for i in range(9):
    keys_tr = pd.Series([f'{i}|{bg}' for bg in bg_tr[i]])
    oof_mean, oof_logcnt, te_map_full, cnt_full = oof_te_by_key(keys_tr, y_tr, m=100.0, prior=prior)
    te_pos_bg_tr[:, i] = oof_mean
    te_pos_bg_logcnt_tr[:, i] = oof_logcnt
    keys_te = pd.Series([f'{i}|{bg}' for bg in bg_te[i]])
    te_vals = keys_te.map(te_map_full).astype('float32').fillna(prior).values
    lc = keys_te.map(cnt_full).fillna(0).astype('int32').values
    te_pos_bg_te[:, i] = te_vals
    te_pos_bg_logcnt_te[:, i] = np.log1p(lc).astype('float32')

# Optional TE for f27_nunique with m=50
keys_nu_tr = pd.Series(f27_nuniq_tr.astype('int16'))
oof_nu_mean, oof_nu_logcnt, nu_map_full, nu_cnt_full = oof_te_by_key(keys_nu_tr, y_tr, m=50.0, prior=prior)
nu_te_tr = oof_nu_mean
nu_logcnt_tr = oof_nu_logcnt
keys_nu_te = pd.Series(f27_nuniq_te.astype('int16'))
nu_te_te = keys_nu_te.map(nu_map_full).astype('float32').fillna(prior).values
nu_logcnt_te = np.log1p(keys_nu_te.map(nu_cnt_full).fillna(0).astype('int32').values).astype('float32')

# Target-free frequencies pooled on train+test
N_all = float(len(train) + len(test))
# pos-char freqs
freq_pos_char_tr = np.zeros((len(train), 10), dtype=np.float32)
freq_pos_char_te = np.zeros((len(test), 10), dtype=np.float32)
for i in range(10):
    keys_all = pd.Series(np.concatenate([np.array([f'{i}|{int(t)}' for t in pos_char_tr[:, i]]), np.array([f'{i}|{int(t)}' for t in pos_char_te[:, i]])]))
    vc = keys_all.value_counts()
    ktr = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[:, i]])
    kte = pd.Series([f'{i}|{int(t)}' for t in pos_char_te[:, i]])
    freq_pos_char_tr[:, i] = (ktr.map(vc).fillna(0).values.astype('float32') / N_all)
    freq_pos_char_te[:, i] = (kte.map(vc).fillna(0).values.astype('float32') / N_all)

# pos-bigram freqs
freq_pos_bg_tr = np.zeros((len(train), 9), dtype=np.float32)
freq_pos_bg_te = np.zeros((len(test), 9), dtype=np.float32)
for i in range(9):
    keys_all = pd.Series(np.concatenate([np.array([f'{i}|{bg}' for bg in bg_tr[i]]), np.array([f'{i}|{bg}' for bg in bg_te[i]])]))
    vc = keys_all.value_counts()
    ktr = pd.Series([f'{i}|{bg}' for bg in bg_tr[i]])
    kte = pd.Series([f'{i}|{bg}' for bg in bg_te[i]])
    freq_pos_bg_tr[:, i] = (ktr.map(vc).fillna(0).values.astype('float32') / N_all)
    freq_pos_bg_te[:, i] = (kte.map(vc).fillna(0).values.astype('float32') / N_all)

# full-string freq
all_full = pd.Series(np.concatenate([s_tr, s_te]))
vc_full = all_full.value_counts()
freq_full_tr = pd.Series(s_tr).map(vc_full).fillna(0).values.astype('float32') / N_all
freq_full_te = pd.Series(s_te).map(vc_full).fillna(0).values.astype('float32') / N_all

# Assemble TE/freq dataframes
te_cols_tr = {}
te_cols_te = {}
for i in range(10):
    te_cols_tr[f'TE_pos_char_{i}_mean'] = te_pos_char_tr[:, i]
    te_cols_te[f'TE_pos_char_{i}_mean'] = te_pos_char_te[:, i]
for i in range(9):
    te_cols_tr[f'TE_pos_bigram_{i}_mean'] = te_pos_bg_tr[:, i]
    te_cols_te[f'TE_pos_bigram_{i}_mean'] = te_pos_bg_te[:, i]
for i in range(10):
    te_cols_tr[f'TE_pos_char_{i}_logcnt'] = te_pos_char_logcnt_tr[:, i]
    te_cols_te[f'TE_pos_char_{i}_logcnt'] = te_pos_char_logcnt_te[:, i]
for i in range(9):
    te_cols_tr[f'TE_pos_bigram_{i}_logcnt'] = te_pos_bg_logcnt_tr[:, i]
    te_cols_te[f'TE_pos_bigram_{i}_logcnt'] = te_pos_bg_logcnt_te[:, i]
te_cols_tr['TE_f27_nunique_mean'] = nu_te_tr
te_cols_te['TE_f27_nunique_mean'] = nu_te_te
te_cols_tr['TE_f27_nunique_logcnt'] = nu_logcnt_tr
te_cols_te['TE_f27_nunique_logcnt'] = nu_logcnt_te
for i in range(10):
    te_cols_tr[f'FREQ_pos_char_{i}'] = freq_pos_char_tr[:, i]
    te_cols_te[f'FREQ_pos_char_{i}'] = freq_pos_char_te[:, i]
for i in range(9):
    te_cols_tr[f'FREQ_pos_bigram_{i}'] = freq_pos_bg_tr[:, i]
    te_cols_te[f'FREQ_pos_bigram_{i}'] = freq_pos_bg_te[:, i]
te_cols_tr['FREQ_full_string'] = freq_full_tr
te_cols_te['FREQ_full_string'] = freq_full_te

te_train = pd.DataFrame(te_cols_tr, index=train.index)
te_test = pd.DataFrame(te_cols_te, index=test.index)

# Build augmented compact feature matrices for unseen model
X_unseen_tr = pd.concat([comp_train.reset_index(drop=True), te_train.reset_index(drop=True)], axis=1)
X_unseen_te = pd.concat([comp_test.reset_index(drop=True), te_test.reset_index(drop=True)], axis=1)
unseen_feature_cols = X_unseen_tr.columns.tolist()

# Sanity checks
assert X_unseen_tr.shape[1] == len(unseen_feature_cols)
assert not np.isnan(X_unseen_tr.values).any(), 'NaNs in X_unseen_tr'
assert not np.isnan(X_unseen_te.values).any(), 'NaNs in X_unseen_te'
log(f'Built TE/freq block. Unseen feature count: {len(unseen_feature_cols)} | time {time.time()-t0:.1f}s')

[2025-09-12 02:07:21] Building OOF target encodings for f_27 positional chars and bigrams, plus frequency encodings...


[2025-09-12 02:08:07] Built TE/freq block. Unseen feature count: 71 | time 46.4s


In [29]:
# Train TE-enhanced unseen models: 6x LightGBM + 1x XGBoost on X_unseen_tr/X_unseen_te
log('Training TE-enhanced unseen models (6 LGB seeds + 1 XGB) ...')
import lightgbm as lgb

assert 'X_unseen_tr' in globals() and 'X_unseen_te' in globals(), 'Run TE feature cell first (X_unseen_tr/X_unseen_te missing)'
X_tr = X_unseen_tr.copy()
X_te = X_unseen_te.copy()
y_tr = train['target'].values.astype('float32')

# Folds (locked)
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

def train_lgb_unseen(seed=42, lr=0.04, num_leaves=224, min_data_in_leaf=240, ff=0.78, bf=0.78, l2=4.0, tag='u42'):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': lr,
        'num_leaves': num_leaves,
        'max_depth': -1,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': ff,
        'bagging_fraction': bf,
        'bagging_freq': 1,
        'lambda_l1': 0.0,
        'lambda_l2': l2,
        'max_bin': 255,
        'verbose': -1,
        'n_jobs': -1,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'data_random_seed': seed
    }
    oof = np.zeros(len(X_tr), dtype='float32')
    ptest = np.zeros(len(X_te), dtype='float32')
    t0 = time.time()
    for fold in range(n_splits):
        trn_idx = np.where(folds_arr != fold)[0]
        val_idx = np.where(folds_arr == fold)[0]
        log(f'[LGB unseen] seed={seed} ff={ff} bf={bf} | Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
        dtr = lgb.Dataset(X_tr.iloc[trn_idx], label=y_tr[trn_idx])
        dvl = lgb.Dataset(X_tr.iloc[val_idx], label=y_tr[val_idx])
        model = lgb.train(params, dtr, num_boost_round=5000, valid_sets=[dtr, dvl], valid_names=['train','valid'], callbacks=[lgb.early_stopping(200), lgb.log_evaluation(200)])
        oof[val_idx] = model.predict(X_tr.iloc[val_idx], num_iteration=model.best_iteration)
        auc = roc_auc_score(y_tr[val_idx], oof[val_idx])
        log(f'[LGB unseen] seed={seed} fold={fold} AUC: {auc:.6f} | best_iter={model.best_iteration}')
        ptest += model.predict(X_te, num_iteration=model.best_iteration) / n_splits
        del dtr, dvl, model; gc.collect()
    cv_auc = roc_auc_score(y_tr, oof)
    log(f'[LGB unseen] seed={seed} OOF AUC: {cv_auc:.6f} | elapsed={time.time()-t0:.1f}s')
    pd.DataFrame({'id': train['id'], f'oof_lgb_unseen_{tag}': oof}).to_csv(f'oof_lgb_unseen_{tag}.csv', index=False)
    pd.DataFrame({'id': test['id'], f'prediction_lgb_unseen_{tag}': ptest}).to_csv(f'pred_lgb_unseen_{tag}.csv', index=False)
    log(f'[LGB unseen] Saved OOF/test preds for {tag}')

# Run 6 LGB seeds with diverse params
lgb_cfgs = [
    # seed, lr, leaves, min_leaf, ff, bf, l2, tag
    (42,   0.040, 224, 240, 0.78, 0.78, 4.0, 'u42'),
    (1337, 0.045, 256, 240, 0.72, 0.72, 5.0, 'u1337'),
    (2025, 0.038, 192, 260, 0.83, 0.75, 3.0, 'u2025'),
    (101,  0.042, 256, 300, 0.70, 0.83, 6.0, 'u101'),
    (999,  0.035, 192, 220, 0.80, 0.70, 4.0, 'u999'),
    (7,    0.040, 240, 260, 0.75, 0.80, 5.0, 'u7'),
]
for s, lr, nl, mdl, ff, bf, l2, tag in lgb_cfgs:
    train_lgb_unseen(seed=s, lr=lr, num_leaves=nl, min_data_in_leaf=mdl, ff=ff, bf=bf, l2=l2, tag=tag)
log('All TE-enhanced LGB unseen seeds finished.')

# XGBoost on TE-enhanced unseen features
log('Training XGBoost unseen (TE-enhanced features)...')
try:
    import xgboost as xgb
except ImportError:
    import sys, subprocess
    log('XGBoost not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb

dtest = xgb.DMatrix(X_te)
oof_xgb_u = np.zeros(len(X_tr), dtype='float32')
pred_xgb_u = np.zeros(len(X_te), dtype='float32')
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'max_bin': 256,
    'eta': 0.045,
    'max_depth': 8,
    'min_child_weight': 80,
    'subsample': 0.80,
    'colsample_bytree': 0.80,
    'lambda': 3.0,
    'nthread': 36,
    'seed': 42
}
num_boost_round = 3500
early_stopping_rounds = 150
for fold in range(n_splits):
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[XGB unseen] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
    dtr = xgb.DMatrix(X_tr.iloc[trn_idx], label=y_tr[trn_idx])
    dvl = xgb.DMatrix(X_tr.iloc[val_idx], label=y_tr[val_idx])
    bst = xgb.train(params=xgb_params, dtrain=dtr, num_boost_round=num_boost_round, evals=[(dtr,'train'),(dvl,'valid')], early_stopping_rounds=early_stopping_rounds, verbose_eval=200)
    best_iter = None
    try:
        attrs = bst.attributes()
        if 'best_iteration' in attrs:
            best_iter = int(attrs['best_iteration'])
    except Exception:
        best_iter = getattr(bst, 'best_iteration', None)
    if best_iter is not None and best_iter >= 0:
        oof_pred = bst.predict(dvl, iteration_range=(0, best_iter + 1))
        pred_te = bst.predict(dtest, iteration_range=(0, best_iter + 1))
    else:
        oof_pred = bst.predict(dvl)
        pred_te = bst.predict(dtest)
    oof_xgb_u[val_idx] = oof_pred.astype('float32')
    pred_xgb_u += (pred_te.astype('float32') / n_splits)
    del dtr, dvl, bst; gc.collect()
auc_xgb_u = roc_auc_score(y_tr, oof_xgb_u)
log(f'[XGB unseen] OOF AUC: {auc_xgb_u:.6f}')
pd.DataFrame({'id': train['id'], 'oof_xgb_unseen': oof_xgb_u}).to_csv('oof_xgb_unseen.csv', index=False)
pd.DataFrame({'id': test['id'], 'prediction_xgb_unseen': pred_xgb_u}).to_csv('pred_xgb_unseen.csv', index=False)
log('[XGB unseen] Saved OOF and test predictions.')

[2025-09-12 02:10:28] Training TE-enhanced unseen models (6 LGB seeds + 1 XGB) ...


[2025-09-12 02:10:28] [LGB unseen] seed=42 ff=0.78 bf=0.78 | Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 200 rounds


[200]	train's auc: 0.833937	valid's auc: 0.817831


[400]	train's auc: 0.848099	valid's auc: 0.819141


...

[600]	train-auc:0.83651	valid-auc:0.80943


[800]	train-auc:0.84098	valid-auc:0.80967


[908]	train-auc:0.84321	valid-auc:0.80964


[2025-09-12 03:13:04] [XGB unseen] OOF AUC: 0.812507


[2025-09-12 03:13:05] [XGB unseen] Saved OOF and test predictions.


In [30]:
# Cell 26: Blend TE-enhanced unseen models and assemble final submission
log('Blending TE-enhanced unseen models and assembling final submission...')
import pandas as pd, numpy as np, os, json
from sklearn.metrics import roc_auc_score

# 1) Load 6x LGB unseen OOF/test preds (TE-enhanced)
def load_oof(path):
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})
def load_pred(path):
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})

lgb_tags = ['u42','u1337','u2025','u101','u999','u7']
oofs = []; preds = []
for tag in lgb_tags:
    po = load_oof(f'oof_lgb_unseen_{tag}.csv')
    pt = load_pred(f'pred_lgb_unseen_{tag}.csv')
    oofs.append(po.rename(columns={'pred': f'oof_{tag}'}))
    preds.append(pt.rename(columns={'pred': f'pred_{tag}'}))

# Merge and average
oof_lgb = oofs[0]
for df in oofs[1:]:
    oof_lgb = oof_lgb.merge(df, on='id', how='inner')
pred_lgb = preds[0]
for df in preds[1:]:
    pred_lgb = pred_lgb.merge(df, on='id', how='inner')
lgb_oof_cols = [c for c in oof_lgb.columns if c != 'id']
lgb_pred_cols = [c for c in pred_lgb.columns if c != 'id']
oof_lgb['lgb_ens'] = oof_lgb[lgb_oof_cols].mean(axis=1).astype('float32')
pred_lgb['lgb_ens'] = pred_lgb[lgb_pred_cols].mean(axis=1).astype('float32')

# 2) Load XGB unseen OOF/test preds (TE-enhanced)
oof_xgb = load_oof('oof_xgb_unseen.csv')
pred_xgb = load_pred('pred_xgb_unseen.csv')

# 3) Tune blend on OOF (global, since no unseen-like rows exist in train)
gt = train[['id','target']].copy()
oof = gt.merge(oof_lgb[['id','lgb_ens']], on='id', how='left').merge(oof_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')

def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / max(len(r) - 1, 1)

oof['lgb_r'] = rank_norm(oof['lgb_ens'])
oof['xgb_r'] = rank_norm(oof['xgb'])

best = {'auc': 0.0, 'mode': 'prob', 'w_lgb': 1.0}
weights = np.round(np.linspace(0.60, 0.95, 8), 2)  # 0.60..0.95
for w in weights:
    prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
    auc_p = roc_auc_score(oof['target'], prob)
    if auc_p > best['auc']:
        best.update({'auc': float(auc_p), 'mode': 'prob', 'w_lgb': float(w)})
    rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
    auc_r = roc_auc_score(oof['target'], rank)
    if auc_r > best['auc']:
        best.update({'auc': float(auc_r), 'mode': 'rank', 'w_lgb': float(w)})
log(f"[TE-unseen blend] Best OOF AUC={best['auc']:.6f} | mode={best['mode']} | w_lgb={best['w_lgb']}")

# 4) Build blended unseen test predictions
sub_te = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left').merge(pred_xgb.rename(columns={'pred':'xgb'}), on='id', how='left')
if best['mode'] == 'prob':
    unseen_pred = best['w_lgb'] * sub_te['lgb_ens'].values + (1 - best['w_lgb']) * sub_te['xgb'].values
else:
    rl = rank_norm(sub_te['lgb_ens']); rx = rank_norm(sub_te['xgb'])
    unseen_pred = best['w_lgb'] * rl + (1 - best['w_lgb']) * rx
unseen_pred = unseen_pred.astype('float32')

# 5) f_27 identity map for seen rows
f27_target_map = train.groupby('f_27')['target'].first().to_dict()
test_seen_mask = test['f_27'].isin(f27_target_map).values
test_unseen_mask = ~test_seen_mask
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_target_map).astype('float32').values

# 6) Final assembly
final = pd.read_csv('sample_submission.csv')[['id']].copy()
final['target'] = np.zeros(len(final), dtype='float32')
final.loc[test_seen_mask, 'target'] = preds_seen
final.loc[test_unseen_mask, 'target'] = unseen_pred[test_unseen_mask]
if final['target'].isna().any():
    raise ValueError('NaNs in final target after assembly')
final['target'] = final['target'].astype('float64').clip(0.0, 1.0)
final.to_csv('submission.csv', index=False)
final.to_csv('submission_te_unseen.csv', index=False)
log('Saved submission.csv using f_27 identity for seen and TE-enhanced blend for unseen.')

# Save meta
with open('blend_unseen_meta_te.json', 'w') as f:
    json.dump(best, f, indent=2)
log(f"Saved blend_unseen_meta_te.json: {best}")

[2025-09-12 03:13:44] Blending TE-enhanced unseen models and assembling final submission...


[2025-09-12 03:13:50] [TE-unseen blend] Best OOF AUC=0.820429 | mode=rank | w_lgb=0.95


[2025-09-12 03:13:53] Saved submission.csv using f_27 identity for seen and TE-enhanced blend for unseen.


[2025-09-12 03:13:53] Saved blend_unseen_meta_te.json: {'auc': 0.8204291485137813, 'mode': 'rank', 'w_lgb': 0.95}


In [31]:
# Cell 27: Guardrails and assertions for f_27 identity map and submission integrity
log('Running guardrails: f_27 purity checks and submission integrity...')
import pandas as pd, numpy as np, json, os

# 1) f_27 -> target purity checks
grp = train.groupby('f_27')['target']
nuniq = grp.nunique()
max_nuniq = int(nuniq.max())
conflicts = int((nuniq > 1).sum())
log(f'f_27 groups: total={len(nuniq)}, max target nunique={max_nuniq}, conflicts={conflicts}')
if max_nuniq > 1 or conflicts > 0:
    # Resolve by majority (mean >= 0.5 -> 1 else 0)
    means = grp.mean()
    f27_map = (means >= 0.5).astype('int8').to_dict()
    log(f'Resolved {conflicts} conflicting f_27 groups by majority vote.')
else:
    # Pure mapping: any aggregation works; use .first() for speed
    f27_map = grp.first().astype('int8').to_dict()

# 2) Seen/unseen masks and coverage
test_seen_mask = test['f_27'].isin(f27_map).values
seen_cnt = int(test_seen_mask.sum())
unseen_cnt = int((~test_seen_mask).sum())
log(f'Seen coverage check -> seen={seen_cnt}, unseen={unseen_cnt}')
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_map)
assert preds_seen.notna().all(), 'NaNs found in seen predictions from identity map'

# 3) Submission integrity
assert os.path.exists('submission.csv'), 'submission.csv missing'
sub = pd.read_csv('submission.csv')
assert list(sub.columns) == ['id','target'], f'Bad submission columns: {list(sub.columns)}'
assert len(sub) == len(test), f'Submission length {len(sub)} != test length {len(test)}'
assert not sub['target'].isna().any(), 'NaNs in submission target'
assert np.isfinite(sub['target']).all(), 'Non-finite values in submission target'
assert sub['target'].between(0.0, 1.0).all(), 'Submission target out of [0,1] bounds'

# 4) If desired, strictly align and resave (idempotent) to be safe
ss = pd.read_csv('sample_submission.csv')[['id']]
sub_aligned = ss.merge(sub, on='id', how='left')
missing = int(sub_aligned['target'].isna().sum())
assert missing == 0, f'Missing predictions for {missing} rows after alignment'
sub_aligned['target'] = sub_aligned['target'].astype('float64').clip(0.0, 1.0)
sub_aligned.to_csv('submission.csv', index=False)
log('Guardrails passed. submission.csv verified and re-saved strictly aligned.')

[2025-09-12 03:16:56] Running guardrails: f_27 purity checks and submission integrity...


[2025-09-12 03:16:57] f_27 groups: total=670919, max target nunique=2, conflicts=41748


[2025-09-12 03:16:57] Resolved 41748 conflicting f_27 groups by majority vote.


[2025-09-12 03:16:58] Seen coverage check -> seen=28435, unseen=71565


[2025-09-12 03:16:58] Guardrails passed. submission.csv verified and re-saved strictly aligned.


In [32]:
# Cell 28: Refined blend + optional meta-stacker for TE-unseen, assemble submission
log('Refining TE-unseen blend (fine grid + optional stacker) and assembling submission...')
import pandas as pd, numpy as np, json, os
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import StratifiedKFold

def load_oof(path):
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id'];
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})
def load_pred(path):
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id'];
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})

# Load unseen OOF/test blocks
lgb_tags = ['u42','u1337','u2025','u101','u999','u7']
oofs = []; preds = []
for tag in lgb_tags:
    oofs.append(load_oof(f'oof_lgb_unseen_{tag}.csv').rename(columns={'pred': f'oof_{tag}'}))
    preds.append(load_pred(f'pred_lgb_unseen_{tag}.csv').rename(columns={'pred': f'pred_{tag}'}))
oof_lgb = oofs[0]
for df in oofs[1:]: oof_lgb = oof_lgb.merge(df, on='id', how='inner')
pred_lgb = preds[0]
for df in preds[1:]: pred_lgb = pred_lgb.merge(df, on='id', how='inner')
lgb_oof_cols = [c for c in oof_lgb.columns if c != 'id']
lgb_pred_cols = [c for c in pred_lgb.columns if c != 'id']
oof_lgb['lgb_ens'] = oof_lgb[lgb_oof_cols].mean(axis=1).astype('float32')
pred_lgb['lgb_ens'] = pred_lgb[lgb_pred_cols].mean(axis=1).astype('float32')

oof_xgb = load_oof('oof_xgb_unseen.csv').rename(columns={'pred':'xgb'})
pred_xgb = load_pred('pred_xgb_unseen.csv').rename(columns={'pred':'xgb'})

gt = train[['id','target']].copy()
oof = gt.merge(oof_lgb[['id','lgb_ens']], on='id', how='left').merge(oof_xgb, on='id', how='left')

def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / max(len(r) - 1, 1)

oof['lgb_r'] = rank_norm(oof['lgb_ens']); oof['xgb_r'] = rank_norm(oof['xgb'])

# Fine grid search in [0.90..0.98] step 0.01 for prob and rank
best = {'auc': 0.0, 'mode': 'prob', 'w_lgb': 1.0}
weights = np.round(np.arange(0.90, 0.981, 0.01), 2)
for w in weights:
    prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
    auc_p = roc_auc_score(oof['target'], prob)
    if auc_p > best['auc']:
        best.update({'auc': float(auc_p), 'mode': 'prob', 'w_lgb': float(w)})
    rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
    auc_r = roc_auc_score(oof['target'], rank)
    if auc_r > best['auc']:
        best.update({'auc': float(auc_r), 'mode': 'rank', 'w_lgb': float(w)})
log(f"[Refine] Best OOF AUC={best['auc']:.6f} | mode={best['mode']} | w_lgb={best['w_lgb']}")

# Optional meta-stacker (logistic on [lgb_ens, xgb]); 10-fold out-of-fold to avoid bias
X_meta = oof[['lgb_ens','xgb']].values.astype('float32')
y_meta = oof['target'].values.astype('int8')
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
oof_stack = np.zeros(len(oof), dtype='float32')
test_stack = np.zeros(len(pred_lgb), dtype='float32')
models = []
for fold, (trn_idx, val_idx) in enumerate(skf.split(X_meta, y_meta), 1):
    Xtr, ytr = X_meta[trn_idx], y_meta[trn_idx]
    Xvl = X_meta[val_idx]
    clf = LogisticRegression(max_iter=1000, solver='lbfgs')
    clf.fit(Xtr, ytr)
    oof_stack[val_idx] = clf.predict_proba(Xvl)[:,1].astype('float32')
    models.append(clf)
auc_stack = roc_auc_score(y_meta, oof_stack)
log(f'[Stacker] Logistic meta OOF AUC={auc_stack:.6f}')
if auc_stack > best['auc'] + 0.0005:
    # Build test stack prediction via average of fold models
    X_test_meta = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left').merge(pred_xgb, on='id', how='left')
    Xt = X_test_meta[['lgb_ens','xgb']].values.astype('float32')
    for m in models:
        test_stack += (m.predict_proba(Xt)[:,1].astype('float32') / len(models))
    chosen = {'mode': 'stack', 'auc': float(auc_stack)}
else:
    chosen = best.copy()

# Build unseen test predictions according to chosen strategy
sub_te = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left').merge(pred_xgb, on='id', how='left')
if chosen.get('mode') == 'stack':
    unseen_pred = test_stack.astype('float32')
else:
    if chosen['mode'] == 'prob':
        unseen_pred = chosen['w_lgb'] * sub_te['lgb_ens'].values + (1 - chosen['w_lgb']) * sub_te['xgb'].values
    else:
        rl = rank_norm(sub_te['lgb_ens']); rx = rank_norm(sub_te['xgb'])
        unseen_pred = chosen['w_lgb'] * rl + (1 - chosen['w_lgb']) * rx
    unseen_pred = unseen_pred.astype('float32')

# Seen mapping with majority resolution guardrail
grp = train.groupby('f_27')['target']
means = grp.mean()
f27_map = (means >= 0.5).astype('int8').to_dict()
test_seen_mask = test['f_27'].isin(f27_map).values
test_unseen_mask = ~test_seen_mask
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_map).astype('float32').values

# Final assembly
final = pd.read_csv('sample_submission.csv')[['id']].copy()
final['target'] = np.zeros(len(final), dtype='float32')
final.loc[test_seen_mask, 'target'] = preds_seen
final.loc[test_unseen_mask, 'target'] = unseen_pred[test_unseen_mask]
if final['target'].isna().any():
    raise ValueError('NaNs in final target after assembly')
final['target'] = final['target'].astype('float64').clip(0.0, 1.0)
final.to_csv('submission.csv', index=False)
final.to_csv('submission_te_unseen_refined.csv', index=False)
meta = {'chosen': chosen, 'grid_best': best}
with open('blend_unseen_meta_te_refined.json','w') as f: json.dump(meta, f, indent=2)
log(f"Saved submission.csv and metadata. Chosen={chosen}")

[2025-09-12 03:18:08] Refining TE-unseen blend (fine grid + optional stacker) and assembling submission...


[2025-09-12 03:18:15] [Refine] Best OOF AUC=0.820443 | mode=rank | w_lgb=0.98


[2025-09-12 03:18:26] [Stacker] Logistic meta OOF AUC=0.820417


[2025-09-12 03:18:28] Saved submission.csv and metadata. Chosen={'auc': 0.8204432246955218, 'mode': 'rank', 'w_lgb': 0.98}


In [33]:
# Cell 29: Pseudo-unseen validation (hold out 20% unique f_27 strings), TE-fit on subset, evaluate unseen AUC
log('Starting pseudo-unseen validation: masking 20% unique f_27, fitting TE on remaining, evaluating held-out...')
import numpy as np, pandas as pd, time, gc, os
from sklearn.metrics import roc_auc_score

rng = np.random.default_rng(42)
s_tr = train['f_27'].astype(str).values
y_tr = train['target'].values.astype('float32')

# 1) Build pseudo-unseen split by f_27 identity
uniq = pd.unique(s_tr)
n_mask = max(1, int(0.20 * len(uniq)))
mask_ids = set(rng.choice(uniq, size=n_mask, replace=False))
holdout_mask = np.array([x in mask_ids for x in s_tr], dtype=bool)
fit_mask = ~holdout_mask
log(f'Pseudo-unseen split: fit_n={int(fit_mask.sum())} | holdout_n={int(holdout_mask.sum())} | uniq_total={len(uniq)} | uniq_masked={len(mask_ids)}')

# 2) Prepare positional tokens
pos_char_tr = np.stack([train[f'f_27_pos_{i}'].astype('int16').values for i in range(10)], axis=1)

def make_bigrams(arr):
    n = arr.shape[0]
    out = [None]*9
    for i in range(9):
        out[i] = np.fromiter((row[i:i+2] for row in arr), count=n, dtype=object)
    return out
bg_tr = make_bigrams(s_tr)

def f27_nunique(arr):
    return np.fromiter((len(set(list(x))) for x in arr), count=len(arr), dtype=np.int16)
f27_nuniq_tr = f27_nunique(s_tr)

# 3) Fit TE maps on fit subset only, apply to fit/holdout
prior = float(train.loc[fit_mask, 'target'].mean())

def fit_te_map(keys_fit, y_fit, m, prior):
    df = pd.DataFrame({'k': keys_fit, 'y': y_fit})
    grp = df.groupby('k')
    cnt = grp['y'].size()
    sumy = grp['y'].sum()
    te = ((sumy + m * prior) / (cnt + m)).astype('float32')
    return te, cnt.astype('int32')

# Positional chars (m=30)
te_pos_char_fit = np.zeros((fit_mask.sum(), 10), dtype=np.float32)
te_pos_char_hold = np.zeros((holdout_mask.sum(), 10), dtype=np.float32)
te_pos_char_logcnt_fit = np.zeros_like(te_pos_char_fit)
te_pos_char_logcnt_hold = np.zeros_like(te_pos_char_hold)
for i in range(10):
    keys_fit = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[fit_mask, i]])
    te_map, cnt_map = fit_te_map(keys_fit.values, y_tr[fit_mask], m=30.0, prior=prior)
    # transform
    k_fit = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[fit_mask, i]])
    k_hld = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[holdout_mask, i]])
    te_pos_char_fit[:, i] = k_fit.map(te_map).astype('float32').fillna(prior).values
    te_pos_char_hold[:, i] = k_hld.map(te_map).astype('float32').fillna(prior).values
    te_pos_char_logcnt_fit[:, i] = np.log1p(k_fit.map(cnt_map).fillna(0).astype('int32').values).astype('float32')
    te_pos_char_logcnt_hold[:, i] = np.log1p(k_hld.map(cnt_map).fillna(0).astype('int32').values).astype('float32')

# Positional bigrams (m=100)
te_pos_bg_fit = np.zeros((fit_mask.sum(), 9), dtype=np.float32)
te_pos_bg_hold = np.zeros((holdout_mask.sum(), 9), dtype=np.float32)
te_pos_bg_logcnt_fit = np.zeros_like(te_pos_bg_fit)
te_pos_bg_logcnt_hold = np.zeros_like(te_pos_bg_hold)
for i in range(9):
    keys_fit = pd.Series([f'{i}|{bg}' for bg in bg_tr[i][fit_mask]])
    te_map, cnt_map = fit_te_map(keys_fit.values, y_tr[fit_mask], m=100.0, prior=prior)
    k_fit = pd.Series([f'{i}|{bg}' for bg in bg_tr[i][fit_mask]])
    k_hld = pd.Series([f'{i}|{bg}' for bg in bg_tr[i][holdout_mask]])
    te_pos_bg_fit[:, i] = k_fit.map(te_map).astype('float32').fillna(prior).values
    te_pos_bg_hold[:, i] = k_hld.map(te_map).astype('float32').fillna(prior).values
    te_pos_bg_logcnt_fit[:, i] = np.log1p(k_fit.map(cnt_map).fillna(0).astype('int32').values).astype('float32')
    te_pos_bg_logcnt_hold[:, i] = np.log1p(k_hld.map(cnt_map).fillna(0).astype('int32').values).astype('float32')

# f27_nunique (m=50)
keys_nu_fit = pd.Series(f27_nuniq_tr[fit_mask].astype('int16'))
te_map_nu, cnt_map_nu = fit_te_map(keys_nu_fit.values, y_tr[fit_mask], m=50.0, prior=prior)
k_fit_nu = pd.Series(f27_nuniq_tr[fit_mask].astype('int16'))
k_hld_nu = pd.Series(f27_nuniq_tr[holdout_mask].astype('int16'))
nu_te_fit = k_fit_nu.map(te_map_nu).astype('float32').fillna(prior).values
nu_te_hold = k_hld_nu.map(te_map_nu).astype('float32').fillna(prior).values
nu_log_fit = np.log1p(k_fit_nu.map(cnt_map_nu).fillna(0).astype('int32').values).astype('float32')
nu_log_hold = np.log1p(k_hld_nu.map(cnt_map_nu).fillna(0).astype('int32').values).astype('float32')

# 4) Target-free frequencies (train+test pooled) - reuse existing approach
N_all = float(len(train) + len(test))
def freq_map(series_all):
    vc = pd.Series(series_all).value_counts()
    return vc
freq_pos_char_fit = np.zeros((fit_mask.sum(), 10), dtype=np.float32)
freq_pos_char_hold = np.zeros((holdout_mask.sum(), 10), dtype=np.float32)
for i in range(10):
    all_keys = np.array([f'{i}|{int(t)}' for t in np.concatenate([train[f'f_27_pos_{i}'].astype('int16').values, test[f'f_27_pos_{i}'].astype('int16').values])])
    vc = pd.Series(all_keys).value_counts()
    kf = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[fit_mask, i]])
    kh = pd.Series([f'{i}|{int(t)}' for t in pos_char_tr[holdout_mask, i]])
    freq_pos_char_fit[:, i] = (kf.map(vc).fillna(0).values.astype('float32') / N_all)
    freq_pos_char_hold[:, i] = (kh.map(vc).fillna(0).values.astype('float32') / N_all)

freq_pos_bg_fit = np.zeros((fit_mask.sum(), 9), dtype=np.float32)
freq_pos_bg_hold = np.zeros((holdout_mask.sum(), 9), dtype=np.float32)
for i in range(9):
    all_keys = np.array([f'{i}|{bg}' for bg in np.concatenate([make_bigrams(train['f_27'].astype(str).values)[i], make_bigrams(test['f_27'].astype(str).values)[i]])])
    vc = pd.Series(all_keys).value_counts()
    kf = pd.Series([f'{i}|{bg}' for bg in bg_tr[i][fit_mask]])
    kh = pd.Series([f'{i}|{bg}' for bg in bg_tr[i][holdout_mask]])
    freq_pos_bg_fit[:, i] = (kf.map(vc).fillna(0).values.astype('float32') / N_all)
    freq_pos_bg_hold[:, i] = (kh.map(vc).fillna(0).values.astype('float32') / N_all)

all_full = pd.Series(np.concatenate([train['f_27'].astype(str).values, test['f_27'].astype(str).values]))
vc_full = all_full.value_counts()
freq_full_fit = pd.Series(s_tr[fit_mask]).map(vc_full).fillna(0).values.astype('float32') / N_all
freq_full_hold = pd.Series(s_tr[holdout_mask]).map(vc_full).fillna(0).values.astype('float32') / N_all

# 5) Assemble feature matrices for fit/holdout using current TE block + compact block
comp_fit = pd.DataFrame(comp_train.loc[fit_mask].reset_index(drop=True))
comp_hold = pd.DataFrame(comp_train.loc[holdout_mask].reset_index(drop=True))
def assemble_block(te_pc, te_pc_lc, te_bg, te_bg_lc, nu_te, nu_log, f_pc, f_bg, f_full, idx_len):
    cols = {}
    for i in range(10): cols[f'TE_pos_char_{i}_mean'] = te_pc[:, i]
    for i in range(9): cols[f'TE_pos_bigram_{i}_mean'] = te_bg[:, i]
    for i in range(10): cols[f'TE_pos_char_{i}_logcnt'] = te_pc_lc[:, i]
    for i in range(9): cols[f'TE_pos_bigram_{i}_logcnt'] = te_bg_lc[:, i]
    cols['TE_f27_nunique_mean'] = nu_te
    cols['TE_f27_nunique_logcnt'] = nu_log
    for i in range(10): cols[f'FREQ_pos_char_{i}'] = f_pc[:, i]
    for i in range(9): cols[f'FREQ_pos_bigram_{i}'] = f_bg[:, i]
    cols['FREQ_full_string'] = f_full
    return pd.DataFrame(cols, index=np.arange(idx_len))

te_fit_df = assemble_block(te_pos_char_fit, te_pos_char_logcnt_fit, te_pos_bg_fit, te_pos_bg_logcnt_fit, nu_te_fit, nu_log_fit, freq_pos_char_fit, freq_pos_bg_fit, freq_full_fit, fit_mask.sum())
te_hold_df = assemble_block(te_pos_char_hold, te_pos_char_logcnt_hold, te_pos_bg_hold, te_pos_bg_logcnt_hold, nu_te_hold, nu_log_hold, freq_pos_char_hold, freq_pos_bg_hold, freq_full_hold, holdout_mask.sum())

X_fit = pd.concat([comp_fit.reset_index(drop=True), te_fit_df.reset_index(drop=True)], axis=1)
X_hold = pd.concat([comp_hold.reset_index(drop=True), te_hold_df.reset_index(drop=True)], axis=1)
y_fit = y_tr[fit_mask]
y_hold = y_tr[holdout_mask]
log(f'Feature blocks ready: X_fit={X_fit.shape}, X_hold={X_hold.shape}')

# 6) Train a quick LightGBM on X_fit and evaluate on X_hold (pseudo-unseen AUC)
import lightgbm as lgb, time
dtr = lgb.Dataset(X_fit, label=y_fit)
params = {
    'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
    'learning_rate': 0.045, 'num_leaves': 224, 'max_depth': -1, 'min_data_in_leaf': 240,
    'feature_fraction': 0.78, 'bagging_fraction': 0.78, 'bagging_freq': 1, 'lambda_l1': 0.0, 'lambda_l2': 4.0,
    'max_bin': 255, 'verbose': -1, 'n_jobs': -1, 'seed': 42
}
t0 = time.time()
model = lgb.train(params, dtr, num_boost_round=2000)
pred_hold = model.predict(X_hold)
auc_hold = roc_auc_score(y_hold, pred_hold)
log(f'[Pseudo-unseen] Holdout AUC={auc_hold:.6f} | train_time={time.time()-t0:.1f}s | fit_n={len(y_fit)} holdout_n={len(y_hold)}')

# Save for reference
pd.DataFrame({'id': train.loc[holdout_mask, 'id'].values, 'oof_pseudo_unseen': pred_hold}).to_csv('oof_pseudo_unseen_holdout.csv', index=False)
with open('pseudo_unseen_meta.json','w') as f:
    json.dump({'holdout_auc': float(auc_hold), 'fit_n': int(len(y_fit)), 'holdout_n': int(len(y_hold))}, f)
gc.collect();

[2025-09-12 03:20:34] Starting pseudo-unseen validation: masking 20% unique f_27, fitting TE on remaining, evaluating held-out...


[2025-09-12 03:20:35] Pseudo-unseen split: fit_n=639929 | holdout_n=160071 | uniq_total=670919 | uniq_masked=134183


[2025-09-12 03:21:18] Feature blocks ready: X_fit=(639929, 71), X_hold=(160071, 71)


[2025-09-12 03:22:28] [Pseudo-unseen] Holdout AUC=0.813502 | train_time=70.1s | fit_n=639929 holdout_n=160071


In [34]:
# Cell 30: Augment TE features for unseen model (trigrams, signatures, non-adj pairs) + target-free patterns; rebuild X_unseen_tr/te
log('Building augmented TE features for unseen model (trigrams, signatures, non-adj pairs) and target-free patterns...')
import numpy as np, pandas as pd, time, gc

t0 = time.time()
assert 'f_27' in train.columns and 'target' in train.columns, 'Missing f_27/target'
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values.astype('int16')
n_splits = int(np.unique(folds_arr).size)
prior = float(train['target'].mean())

# Strings
s_tr = train['f_27'].astype(str).values
s_te = test['f_27'].astype(str).values

# Positional char int tokens (10)
pos_char_tr = np.stack([train[f'f_27_pos_{i}'].astype('int16').values for i in range(10)], axis=1)
pos_char_te = np.stack([test[f'f_27_pos_{i}'].astype('int16').values for i in range(10)], axis=1)

# Bigrams arrays (string), for reuse if needed
def make_bigrams(arr):
    n = arr.shape[0]
    out = [None]*9
    for i in range(9):
        out[i] = np.fromiter((row[i:i+2] for row in arr), count=n, dtype=object)
    return out
bg_tr = make_bigrams(s_tr)
bg_te = make_bigrams(s_te)

# Trigrams arrays (string), i=0..7
def make_trigrams(arr):
    n = arr.shape[0]
    out = [None]*8
    for i in range(8):
        out[i] = np.fromiter((row[i:i+3] for row in arr), count=n, dtype=object)
    return out
tri_tr = make_trigrams(s_tr)
tri_te = make_trigrams(s_te)

# Helper signatures
def run_length_signature(s):
    # returns tuple of consecutive equal run lengths, e.g., 'AAABB' -> (3,2)
    res = []
    cur = 1
    for i in range(1, len(s)):
        if s[i] == s[i-1]: cur += 1
        else:
            res.append(cur); cur = 1
    res.append(cur)
    return tuple(res)

def count_hist_signature(s):
    # multiset of per-char counts sorted, e.g., 'AAABBC' -> (3,2,1)
    from collections import Counter
    cnt = Counter(s)
    return tuple(sorted(cnt.values(), reverse=True))

def majority_char_with_count(s):
    from collections import Counter
    cnt = Counter(s)
    ch, c = max(cnt.items(), key=lambda kv: kv[1])
    return f'{ch}|{c}'

# Non-adjacent symmetric pairs around center
pairs = [(0,9),(1,8),(2,7),(3,6),(4,5)]

# Helper: OOF TE by key with smoothing m; returns oof_mean, oof_logcnt, test_mean, test_logcnt
def oof_te_by_key(keys_tr, y, keys_te, m, prior):
    keys_tr = pd.Series(keys_tr)
    oof_vals = np.zeros(len(keys_tr), dtype=np.float32)
    oof_logcnt = np.zeros(len(keys_tr), dtype=np.float32)
    for fold in range(n_splits):
        trn_idx = np.where(folds_arr != fold)[0]
        val_idx = np.where(folds_arr == fold)[0]
        k_tr = keys_tr.iloc[trn_idx]
        y_tr = y[trn_idx]
        grp = pd.DataFrame({'k': k_tr.values, 'y': y_tr}).groupby('k')
        cnt = grp['y'].size()
        sumy = grp['y'].sum()
        te_map = ((sumy + m * prior) / (cnt + m)).astype('float32')
        lc_map = cnt.astype('int32')
        kval = keys_tr.iloc[val_idx]
        te = kval.map(te_map).astype('float32')
        lc = kval.map(lc_map).fillna(0).astype('int32')
        te = te.fillna(prior).values
        oof_vals[val_idx] = te
        oof_logcnt[val_idx] = np.log1p(lc.values).astype('float32')
    # Full fit for test
    grp_full = pd.DataFrame({'k': keys_tr.values, 'y': y}).groupby('k')
    cnt_full = grp_full['y'].size()
    sumy_full = grp_full['y'].sum()
    te_map_full = ((sumy_full + m * prior) / (cnt_full + m)).astype('float32')
    lc_full = cnt_full.astype('int32')
    keys_te = pd.Series(keys_te)
    te_te = keys_te.map(te_map_full).astype('float32').fillna(prior).values
    lc_te = np.log1p(keys_te.map(lc_full).fillna(0).astype('int32').values).astype('float32')
    return oof_vals, oof_logcnt, te_te, lc_te

y_tr = train['target'].values.astype('float32')

# 1) Recompute existing TEs with adjusted smoothing (pos-char m~25, bigram m~70)
te_pos_char_tr_mean = np.zeros((len(train), 10), dtype=np.float32)
te_pos_char_tr_log = np.zeros((len(train), 10), dtype=np.float32)
te_pos_char_te_mean = np.zeros((len(test), 10), dtype=np.float32)
te_pos_char_te_log = np.zeros((len(test), 10), dtype=np.float32)
for i in range(10):
    k_tr = np.array([f'{i}|{int(t)}' for t in pos_char_tr[:, i]], dtype=object)
    k_te = np.array([f'{i}|{int(t)}' for t in pos_char_te[:, i]], dtype=object)
    o_m, o_l, t_m, t_l = oof_te_by_key(k_tr, y_tr, k_te, m=25.0, prior=prior)
    te_pos_char_tr_mean[:, i] = o_m; te_pos_char_tr_log[:, i] = o_l
    te_pos_char_te_mean[:, i] = t_m; te_pos_char_te_log[:, i] = t_l

te_pos_bg_tr_mean = np.zeros((len(train), 9), dtype=np.float32)
te_pos_bg_tr_log = np.zeros((len(train), 9), dtype=np.float32)
te_pos_bg_te_mean = np.zeros((len(test), 9), dtype=np.float32)
te_pos_bg_te_log = np.zeros((len(test), 9), dtype=np.float32)
for i in range(9):
    k_tr = np.array([f'{i}|{bg}' for bg in bg_tr[i]], dtype=object)
    k_te = np.array([f'{i}|{bg}' for bg in bg_te[i]], dtype=object)
    o_m, o_l, t_m, t_l = oof_te_by_key(k_tr, y_tr, k_te, m=70.0, prior=prior)
    te_pos_bg_tr_mean[:, i] = o_m; te_pos_bg_tr_log[:, i] = o_l
    te_pos_bg_te_mean[:, i] = t_m; te_pos_bg_te_log[:, i] = t_l

# f27_nunique TE (m=50 unchanged)
def f27_nunique(arr):
    return np.fromiter((len(set(list(x))) for x in arr), count=len(arr), dtype=np.int16)
f27_nu_tr = f27_nunique(s_tr)
f27_nu_te = f27_nunique(s_te)
k_tr_nu = f27_nu_tr.astype('int16')
k_te_nu = f27_nu_te.astype('int16')
o_m, o_l, t_m, t_l = oof_te_by_key(k_tr_nu, y_tr, k_te_nu, m=50.0, prior=prior)
nu_te_tr_mean = o_m; nu_te_tr_log = o_l
nu_te_te_mean = t_m; nu_te_te_log = t_l

# 2) Positional trigrams TE (critical): i=0..7, m=180–200
te_tri_tr_mean = np.zeros((len(train), 8), dtype=np.float32)
te_tri_tr_log = np.zeros((len(train), 8), dtype=np.float32)
te_tri_te_mean = np.zeros((len(test), 8), dtype=np.float32)
te_tri_te_log = np.zeros((len(test), 8), dtype=np.float32)
for i in range(8):
    ktr = np.array([f'{i}|{tri}' for tri in tri_tr[i]], dtype=object)
    kte = np.array([f'{i}|{tri}' for tri in tri_te[i]], dtype=object)
    o_m, o_l, t_m, t_l = oof_te_by_key(ktr, y_tr, kte, m=180.0, prior=prior)
    te_tri_tr_mean[:, i] = o_m; te_tri_tr_log[:, i] = o_l
    te_tri_te_mean[:, i] = t_m; te_tri_te_log[:, i] = t_l

# 3) Count-hist signature TE (sorted multiset of char counts), m ~ 110
k_tr_hist = np.fromiter((str(count_hist_signature(s)) for s in s_tr), count=len(s_tr), dtype=object)
k_te_hist = np.fromiter((str(count_hist_signature(s)) for s in s_te), count=len(s_te), dtype=object)
hist_tr_mean, hist_tr_log, hist_te_mean, hist_te_log = oof_te_by_key(k_tr_hist, y_tr, k_te_hist, m=110.0, prior=prior)

# 4) Run-length signature TE, m ~ 70
k_tr_run = np.fromiter((str(run_length_signature(s)) for s in s_tr), count=len(s_tr), dtype=object)
k_te_run = np.fromiter((str(run_length_signature(s)) for s in s_te), count=len(s_te), dtype=object)
run_tr_mean, run_tr_log, run_te_mean, run_te_log = oof_te_by_key(k_tr_run, y_tr, k_te_run, m=70.0, prior=prior)

# 5) Selected non-adjacent symmetric pairs TE, m ~ 100
pair_keys_tr = {}
pair_keys_te = {}
for (i,j) in pairs:
    pair_keys_tr[(i,j)] = np.fromiter((f'{i}_{j}|{row[i]}{row[j]}' for row in s_tr), count=len(s_tr), dtype=object)
    pair_keys_te[(i,j)] = np.fromiter((f'{i}_{j}|{row[i]}{row[j]}' for row in s_te), count=len(s_te), dtype=object)
pair_tr_mean = {}; pair_tr_log = {}; pair_te_mean = {}; pair_te_log = {}
for (i,j), ktr in pair_keys_tr.items():
    kte = pair_keys_te[(i,j)]
    o_m, o_l, t_m, t_l = oof_te_by_key(ktr, y_tr, kte, m=100.0, prior=prior)
    pair_tr_mean[(i,j)] = o_m; pair_tr_log[(i,j)] = o_l
    pair_te_mean[(i,j)] = t_m; pair_te_log[(i,j)] = t_l

# 6) Optional: Majority-char with count TE, m ~ 60
k_tr_maj = np.fromiter((majority_char_with_count(s) for s in s_tr), count=len(s_tr), dtype=object)
k_te_maj = np.fromiter((majority_char_with_count(s) for s in s_te), count=len(s_te), dtype=object)
maj_tr_mean, maj_tr_log, maj_te_mean, maj_te_log = oof_te_by_key(k_tr_maj, y_tr, k_te_maj, m=60.0, prior=prior)

# Target-free frequencies (keep from previous block; recompute minimal essentials)
N_all = float(len(train) + len(test))
freq_full_tr = pd.Series(s_tr).map(pd.Series(np.concatenate([s_tr, s_te])).value_counts()).fillna(0).values.astype('float32') / N_all
freq_full_te = pd.Series(s_te).map(pd.Series(np.concatenate([s_tr, s_te])).value_counts()).fillna(0).values.astype('float32') / N_all

# Target-free pattern features
def position_of_first_repeat(s):
    for i in range(1, len(s)):
        if s[i] == s[i-1]: return i
    return 10
def alternating_score(s):
    diffs = sum(1 for i in range(1, len(s)) if s[i] != s[i-1])
    return diffs / 9.0
def same_diff_counts(s):
    same = sum(1 for i in range(1, len(s)) if s[i] == s[i-1])
    diff = 9 - same
    return same, diff

pos_first_rep_tr = np.fromiter((position_of_first_repeat(x) for x in s_tr), count=len(s_tr), dtype=np.int16)
pos_first_rep_te = np.fromiter((position_of_first_repeat(x) for x in s_te), count=len(s_te), dtype=np.int16)
alt_score_tr = np.fromiter((alternating_score(x) for x in s_tr), count=len(s_tr), dtype=np.float32)
alt_score_te = np.fromiter((alternating_score(x) for x in s_te), count=len(s_te), dtype=np.float32)
same_tr = np.zeros(len(s_tr), dtype=np.int8); diff_tr = np.zeros(len(s_tr), dtype=np.int8)
same_te = np.zeros(len(s_te), dtype=np.int8); diff_te = np.zeros(len(s_te), dtype=np.int8)
for idx, x in enumerate(s_tr):
    a,b = same_diff_counts(x); same_tr[idx] = a; diff_tr[idx] = b
for idx, x in enumerate(s_te):
    a,b = same_diff_counts(x); same_te[idx] = a; diff_te[idx] = b
num_runs_tr = (diff_tr + 1).astype('int8')
num_runs_te = (diff_te + 1).astype('int8')

# Include existing robust f_27 summaries if present
add_exist_cols = {}
for col in ['f_27_longest_run','f_27_transitions','f27_entropy','f27_first_last_same','f27_pal_matches','f27_majority_cnt','f27_majority_idx']:
    if col in train.columns and col in test.columns:
        add_exist_cols[col+'_tr'] = train[col].values
        add_exist_cols[col+'_te'] = test[col].values

# Assemble TE/freq/derived dataframes
te_cols_tr = {}
te_cols_te = {}
# Existing adjusted TEs
for i in range(10):
    te_cols_tr[f'TE_pos_char_{i}_mean'] = te_pos_char_tr_mean[:, i]
    te_cols_te[f'TE_pos_char_{i}_mean'] = te_pos_char_te_mean[:, i]
    te_cols_tr[f'TE_pos_char_{i}_logcnt'] = te_pos_char_tr_log[:, i]
    te_cols_te[f'TE_pos_char_{i}_logcnt'] = te_pos_char_te_log[:, i]
for i in range(9):
    te_cols_tr[f'TE_pos_bigram_{i}_mean'] = te_pos_bg_tr_mean[:, i]
    te_cols_te[f'TE_pos_bigram_{i}_mean'] = te_pos_bg_te_mean[:, i]
    te_cols_tr[f'TE_pos_bigram_{i}_logcnt'] = te_pos_bg_tr_log[:, i]
    te_cols_te[f'TE_pos_bigram_{i}_logcnt'] = te_pos_bg_te_log[:, i]
te_cols_tr['TE_f27_nunique_mean'] = nu_te_tr_mean
te_cols_te['TE_f27_nunique_mean'] = nu_te_te_mean
te_cols_tr['TE_f27_nunique_logcnt'] = nu_te_tr_log
te_cols_te['TE_f27_nunique_logcnt'] = nu_te_te_log
# New: trigrams
for i in range(8):
    te_cols_tr[f'TE_pos_trigram_{i}_mean'] = te_tri_tr_mean[:, i]
    te_cols_te[f'TE_pos_trigram_{i}_mean'] = te_tri_te_mean[:, i]
    te_cols_tr[f'TE_pos_trigram_{i}_logcnt'] = te_tri_tr_log[:, i]
    te_cols_te[f'TE_pos_trigram_{i}_logcnt'] = te_tri_te_log[:, i]
# New: count-hist and run-length signatures
te_cols_tr['TE_counthist_mean'] = hist_tr_mean
te_cols_tr['TE_counthist_logcnt'] = hist_tr_log
te_cols_te['TE_counthist_mean'] = hist_te_mean
te_cols_te['TE_counthist_logcnt'] = hist_te_log
te_cols_tr['TE_runlen_mean'] = run_tr_mean
te_cols_tr['TE_runlen_logcnt'] = run_tr_log
te_cols_te['TE_runlen_mean'] = run_te_mean
te_cols_te['TE_runlen_logcnt'] = run_te_log
# New: non-adjacent pairs
for (i,j) in pairs:
    key = f'TE_pair_{i}_{j}'
    te_cols_tr[f'{key}_mean'] = pair_tr_mean[(i,j)]
    te_cols_tr[f'{key}_logcnt'] = pair_tr_log[(i,j)]
    te_cols_te[f'{key}_mean'] = pair_te_mean[(i,j)]
    te_cols_te[f'{key}_logcnt'] = pair_te_log[(i,j)]
# Optional: majority-char TE
te_cols_tr['TE_majority_charcnt_mean'] = maj_tr_mean
te_cols_tr['TE_majority_charcnt_logcnt'] = maj_tr_log
te_cols_te['TE_majority_charcnt_mean'] = maj_te_mean
te_cols_te['TE_majority_charcnt_logcnt'] = maj_te_log
# Target-free core freq
te_cols_tr['FREQ_full_string'] = freq_full_tr
te_cols_te['FREQ_full_string'] = freq_full_te
# Target-free derived
te_cols_tr['pos_first_repeat'] = pos_first_rep_tr.astype('int16')
te_cols_te['pos_first_repeat'] = pos_first_rep_te.astype('int16')
te_cols_tr['alt_score'] = alt_score_tr.astype('float32')
te_cols_te['alt_score'] = alt_score_te.astype('float32')
te_cols_tr['same_adj'] = same_tr.astype('int8')
te_cols_te['same_adj'] = same_te.astype('int8')
te_cols_tr['diff_adj'] = diff_tr.astype('int8')
te_cols_te['diff_adj'] = diff_te.astype('int8')
te_cols_tr['num_runs'] = num_runs_tr.astype('int8')
te_cols_te['num_runs'] = num_runs_te.astype('int8')
# Attach existing robust summaries if present
for col in ['f_27_longest_run','f_27_transitions','f27_entropy','f27_first_last_same','f27_pal_matches','f27_majority_cnt','f27_majority_idx']:
    if col in train.columns and col in test.columns:
        te_cols_tr[col] = train[col].values
        te_cols_te[col] = test[col].values

te_train_aug = pd.DataFrame(te_cols_tr, index=train.index)
te_test_aug = pd.DataFrame(te_cols_te, index=test.index)

# Concatenate with compact features block for unseen
assert 'comp_train' in globals() and 'comp_test' in globals(), 'Run pivot prep to build compact features first'
X_unseen_tr = pd.concat([comp_train.reset_index(drop=True), te_train_aug.reset_index(drop=True)], axis=1)
X_unseen_te = pd.concat([comp_test.reset_index(drop=True), te_test_aug.reset_index(drop=True)], axis=1)
unseen_feature_cols = X_unseen_tr.columns.tolist()

# Sanity checks
assert X_unseen_tr.shape[0] == len(train) and X_unseen_te.shape[0] == len(test)
assert not np.isnan(X_unseen_tr.values).any(), 'NaNs in X_unseen_tr'
assert not np.isnan(X_unseen_te.values).any(), 'NaNs in X_unseen_te'
log(f'Augmented unseen features built: {X_unseen_tr.shape[1]} cols | time {time.time()-t0:.1f}s')
gc.collect();

[2025-09-12 03:27:58] Building augmented TE features for unseen model (trigrams, signatures, non-adj pairs) and target-free patterns...


[2025-09-12 03:29:02] Augmented unseen features built: 96 cols | time 64.6s


In [35]:
# Cell 31: Retrain unseen models on augmented features (4 LGB seeds + 1 XGB) with tightened params
log('Training augmented TE unseen models: 4x LGB + 1x XGB ...')
import numpy as np, pandas as pd, time, gc
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

assert 'X_unseen_tr' in globals() and 'X_unseen_te' in globals(), 'Run Cell 30 to build augmented features first'
X_tr = X_unseen_tr.copy()
X_te = X_unseen_te.copy()
y_tr = train['target'].values.astype('float32')

# Folds (locked, same as before)
folds_df = pd.read_csv('folds_10fold_seed42.csv')
folds_arr = folds_df['fold'].values
n_splits = len(np.unique(folds_arr))

def train_lgb_unseen_aug(seed=42, lr=0.040, num_leaves=256, min_data_in_leaf=300, ff=0.74, bf=0.80, l2=6.0, tag='au_s42'):
    params = {
        'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
        'learning_rate': lr, 'num_leaves': num_leaves, 'max_depth': -1,
        'min_data_in_leaf': min_data_in_leaf, 'feature_fraction': ff, 'bagging_fraction': bf, 'bagging_freq': 1,
        'lambda_l1': 0.0, 'lambda_l2': l2, 'max_bin': 255, 'verbose': -1, 'n_jobs': -1,
        'seed': seed, 'feature_fraction_seed': seed, 'bagging_seed': seed, 'data_random_seed': seed
    }
    oof = np.zeros(len(X_tr), dtype='float32')
    ptest = np.zeros(len(X_te), dtype='float32')
    t0 = time.time()
    for fold in range(n_splits):
        trn_idx = np.where(folds_arr != fold)[0]
        val_idx = np.where(folds_arr == fold)[0]
        log(f'[LGB unseen AUG] seed={seed} ff={ff} bf={bf} | Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
        dtr = lgb.Dataset(X_tr.iloc[trn_idx], label=y_tr[trn_idx])
        dvl = lgb.Dataset(X_tr.iloc[val_idx], label=y_tr[val_idx])
        model = lgb.train(params, dtr, num_boost_round=6000, valid_sets=[dtr, dvl], valid_names=['train','valid'], callbacks=[lgb.early_stopping(200), lgb.log_evaluation(200)])
        oof[val_idx] = model.predict(X_tr.iloc[val_idx], num_iteration=model.best_iteration)
        auc = roc_auc_score(y_tr[val_idx], oof[val_idx])
        log(f'[LGB unseen AUG] seed={seed} fold={fold} AUC: {auc:.6f} | best_iter={model.best_iteration}')
        ptest += model.predict(X_te, num_iteration=model.best_iteration) / n_splits
        del dtr, dvl, model; gc.collect()
    cv_auc = roc_auc_score(y_tr, oof)
    log(f'[LGB unseen AUG] seed={seed} OOF AUC: {cv_auc:.6f} | elapsed={time.time()-t0:.1f}s')
    pd.DataFrame({'id': train['id'], f'oof_lgb_unseen_{tag}': oof}).to_csv(f'oof_lgb_unseen_{tag}.csv', index=False)
    pd.DataFrame({'id': test['id'], f'prediction_lgb_unseen_{tag}': ptest}).to_csv(f'pred_lgb_unseen_{tag}.csv', index=False)
    log(f'[LGB unseen AUG] Saved OOF/test preds for {tag}')

# 4 LGB seeds with slight diversity per expert guidance
lgb_cfgs = [
    (42,   0.040, 256, 300, 0.74, 0.82, 6.0, 'au_s42'),
    (1337, 0.042, 288, 320, 0.70, 0.85, 7.0, 'au_s1337'),
    (2025, 0.038, 224, 300, 0.78, 0.78, 5.0, 'au_s2025'),
    (7,    0.040, 320, 340, 0.72, 0.80, 8.0, 'au_s7'),
]
for s, lr, nl, mdl, ff, bf, l2, tag in lgb_cfgs:
    train_lgb_unseen_aug(seed=s, lr=lr, num_leaves=nl, min_data_in_leaf=mdl, ff=ff, bf=bf, l2=l2, tag=tag)
log('All augmented LGB unseen seeds finished.')

# XGBoost on augmented features
log('Training XGBoost unseen (augmented features) ...')
try:
    import xgboost as xgb
except ImportError:
    import sys, subprocess
    log('XGBoost not found. Installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb

dtest = xgb.DMatrix(X_te)
oof_xgb_u = np.zeros(len(X_tr), dtype='float32')
pred_xgb_u = np.zeros(len(X_te), dtype='float32')
xgb_params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc', 'tree_method': 'hist', 'max_bin': 256,
    'eta': 0.040, 'max_depth': 9, 'min_child_weight': 120, 'subsample': 0.80, 'colsample_bytree': 0.75, 'lambda': 4.0,
    'nthread': 36, 'seed': 42
}
num_boost_round = 3800
early_stopping_rounds = 180
for fold in range(n_splits):
    trn_idx = np.where(folds_arr != fold)[0]
    val_idx = np.where(folds_arr == fold)[0]
    log(f'[XGB unseen AUG] Fold {fold+1}/{n_splits} | trn={len(trn_idx)} val={len(val_idx)}')
    dtr = xgb.DMatrix(X_tr.iloc[trn_idx], label=y_tr[trn_idx])
    dvl = xgb.DMatrix(X_tr.iloc[val_idx], label=y_tr[val_idx])
    bst = xgb.train(params=xgb_params, dtrain=dtr, num_boost_round=num_boost_round, evals=[(dtr,'train'),(dvl,'valid')], early_stopping_rounds=early_stopping_rounds, verbose_eval=200)
    best_iter = None
    try:
        attrs = bst.attributes()
        if 'best_iteration' in attrs:
            best_iter = int(attrs['best_iteration'])
    except Exception:
        best_iter = getattr(bst, 'best_iteration', None)
    if best_iter is not None and best_iter >= 0:
        oof_pred = bst.predict(dvl, iteration_range=(0, best_iter + 1))
        pred_te = bst.predict(dtest, iteration_range=(0, best_iter + 1))
    else:
        oof_pred = bst.predict(dvl)
        pred_te = bst.predict(dtest)
    oof_xgb_u[val_idx] = oof_pred.astype('float32')
    pred_xgb_u += (pred_te.astype('float32') / n_splits)
    del dtr, dvl, bst; gc.collect()
auc_xgb_u = roc_auc_score(y_tr, oof_xgb_u)
log(f'[XGB unseen AUG] OOF AUC: {auc_xgb_u:.6f}')
pd.DataFrame({'id': train['id'], 'oof_xgb_unseen_aug': oof_xgb_u}).to_csv('oof_xgb_unseen_aug.csv', index=False)
pd.DataFrame({'id': test['id'], 'prediction_xgb_unseen_aug': pred_xgb_u}).to_csv('pred_xgb_unseen_aug.csv', index=False)
log('[XGB unseen AUG] Saved OOF and test predictions.')

[2025-09-12 03:30:31] Training augmented TE unseen models: 4x LGB + 1x XGB ...


[2025-09-12 03:30:31] [LGB unseen AUG] seed=42 ff=0.74 bf=0.82 | Fold 1/10 | trn=720000 val=80000


Training until validation scores don't improve for 200 rounds


[200]	train's auc: 0.837319	valid's auc: 0.81735


[400]	train's auc: 0.854422	valid's auc: 0.818343


...

[800]	train-auc:0.84435	valid-auc:0.81565


[1000]	train-auc:0.84910	valid-auc:0.81562


[1090]	train-auc:0.85113	valid-auc:0.81556


[2025-09-12 04:20:00] [XGB unseen AUG] OOF AUC: 0.814888


[2025-09-12 04:20:01] [XGB unseen AUG] Saved OOF and test predictions.


In [36]:
# Cell 32: Blend augmented unseen models (4 LGB au_* + XGB aug) and build alternative submission
log('Blending augmented unseen models (au_*) and creating alternative submission...')
import pandas as pd, numpy as np, json, os
from sklearn.metrics import roc_auc_score

def load_oof(path):
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})
def load_pred(path):
    df = pd.read_csv(path); cols = [c for c in df.columns if c != 'id']
    return df[['id', cols[0]]].rename(columns={cols[0]: 'pred'})

# Load augmented LGB OOF/test
au_tags = ['au_s42','au_s1337','au_s2025','au_s7']
oofs = []; preds = []
for tag in au_tags:
    oofs.append(load_oof(f'oof_lgb_unseen_{tag}.csv').rename(columns={'pred': f'oof_{tag}'}))
    preds.append(load_pred(f'pred_lgb_unseen_{tag}.csv').rename(columns={'pred': f'pred_{tag}'}))
oof_lgb = oofs[0]
for df in oofs[1:]: oof_lgb = oof_lgb.merge(df, on='id', how='inner')
pred_lgb = preds[0]
for df in preds[1:]: pred_lgb = pred_lgb.merge(df, on='id', how='inner')
lgb_oof_cols = [c for c in oof_lgb.columns if c != 'id']
lgb_pred_cols = [c for c in pred_lgb.columns if c != 'id']
oof_lgb['lgb_ens'] = oof_lgb[lgb_oof_cols].mean(axis=1).astype('float32')
pred_lgb['lgb_ens'] = pred_lgb[lgb_pred_cols].mean(axis=1).astype('float32')

# Load augmented XGB OOF/test
oof_xgb = load_oof('oof_xgb_unseen_aug.csv').rename(columns={'pred':'xgb'})
pred_xgb = load_pred('pred_xgb_unseen_aug.csv').rename(columns={'pred':'xgb'})

# OOF blend search (prob vs rank) with weights in [0.85..0.96]
gt = train[['id','target']].copy()
oof = gt.merge(oof_lgb[['id','lgb_ens']], on='id', how='left').merge(oof_xgb, on='id', how='left')
def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / max(len(r) - 1, 1)
oof['lgb_r'] = rank_norm(oof['lgb_ens']); oof['xgb_r'] = rank_norm(oof['xgb'])
best = {'auc': 0.0, 'mode': 'prob', 'w_lgb': 1.0}
for w in np.round(np.arange(0.85, 0.961, 0.01), 2):
    prob = w * oof['lgb_ens'] + (1 - w) * oof['xgb']
    ap = roc_auc_score(oof['target'], prob)
    if ap > best['auc']: best.update({'auc': float(ap), 'mode': 'prob', 'w_lgb': float(w)})
    rank = w * oof['lgb_r'] + (1 - w) * oof['xgb_r']
    ar = roc_auc_score(oof['target'], rank)
    if ar > best['auc']: best.update({'auc': float(ar), 'mode': 'rank', 'w_lgb': float(w)})
log(f"[Aug unseen blend] Best OOF AUC={best['auc']:.6f} | mode={best['mode']} | w_lgb={best['w_lgb']}")

# Build unseen test predictions according to best
sub_te = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left').merge(pred_xgb, on='id', how='left')
if best['mode'] == 'prob':
    unseen_pred = best['w_lgb'] * sub_te['lgb_ens'].values + (1 - best['w_lgb']) * sub_te['xgb'].values
else:
    rl = rank_norm(sub_te['lgb_ens']); rx = rank_norm(sub_te['xgb'])
    unseen_pred = best['w_lgb'] * rl + (1 - best['w_lgb']) * rx
unseen_pred = unseen_pred.astype('float32')

# Seen mapping via majority resolution (guardrail)
grp = train.groupby('f_27')['target']; means = grp.mean(); f27_map = (means >= 0.5).astype('int8').to_dict()
test_seen_mask = test['f_27'].isin(f27_map).values
test_unseen_mask = ~test_seen_mask
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_map).astype('float32').values

# Assemble alternative submission (do not overwrite submission.csv)
final_alt = pd.read_csv('sample_submission.csv')[['id']].copy()
final_alt['target'] = np.zeros(len(final_alt), dtype='float32')
final_alt.loc[test_seen_mask, 'target'] = preds_seen
final_alt.loc[test_unseen_mask, 'target'] = unseen_pred[test_unseen_mask]
final_alt['target'] = final_alt['target'].astype('float64').clip(0.0, 1.0)
final_alt.to_csv('submission_unseen_aug.csv', index=False)
with open('blend_unseen_meta_aug.json','w') as f: json.dump(best, f, indent=2)
log('Saved submission_unseen_aug.csv with augmented unseen blend and saved metadata.')

[2025-09-12 04:20:58] Blending augmented unseen models (au_*) and creating alternative submission...


[2025-09-12 04:21:06] [Aug unseen blend] Best OOF AUC=0.819963 | mode=rank | w_lgb=0.96


[2025-09-12 04:21:08] Saved submission_unseen_aug.csv with augmented unseen blend and saved metadata.


In [37]:
# Cell 33: Optional rarity rank post-process on current TE-unseen blend
log('Applying rarity rank post-process on current TE-unseen blend (alpha in {0.95,0.97,0.99})...')
import pandas as pd, numpy as np, json

# Load current TE-unseen blended test preds (from Cell 28's refined best) or rebuild quickly
pred_lgb = pd.read_csv('pred_lgb_unseen_u42.csv');
for tag in ['u1337','u2025','u101','u999','u7']:
    df = pd.read_csv(f'pred_lgb_unseen_{tag}.csv')
    col = [c for c in df.columns if c != 'id'][0]
    pred_lgb = pred_lgb.merge(df[['id', col]].rename(columns={col: f'pred_{tag}'}), on='id', how='inner')
lgb_cols = [c for c in pred_lgb.columns if c != 'id']
pred_lgb['lgb_ens'] = pred_lgb[lgb_cols].mean(axis=1).astype('float32')
pred_xgb = pd.read_csv('pred_xgb_unseen.csv')
xcol = [c for c in pred_xgb.columns if c != 'id'][0]
pred_xgb = pred_xgb.rename(columns={xcol: 'xgb'})
sub_blend = pd.read_csv('sample_submission.csv')[['id']].merge(pred_lgb[['id','lgb_ens']], on='id', how='left').merge(pred_xgb, on='id', how='left')

# Use refined best weight/mode from meta if available; else fallback to rank w=0.96
w_meta = {'mode': 'rank', 'w_lgb': 0.96}
try:
    with open('blend_unseen_meta_te_refined.json', 'r') as f:
        meta = json.load(f)
        if 'chosen' in meta:
            w_meta = meta['chosen']
        elif 'mode' in meta and 'w_lgb' in meta:
            w_meta = {'mode': meta['mode'], 'w_lgb': meta['w_lgb']}
except Exception:
    pass

def rank_norm(x):
    r = pd.Series(x).rank(method='average').values
    return (r - 1) / max(len(r) - 1, 1)

if w_meta.get('mode','rank') == 'prob':
    base_pred = (w_meta['w_lgb'] * sub_blend['lgb_ens'].values + (1 - w_meta['w_lgb']) * sub_blend['xgb'].values).astype('float32')
else:
    rl = rank_norm(sub_blend['lgb_ens']); rx = rank_norm(sub_blend['xgb'])
    base_pred = (w_meta['w_lgb'] * rl + (1 - w_meta['w_lgb']) * rx).astype('float32')

# Build 1 - FREQ_full_string rarity score (train+test pooled)
s_tr = train['f_27'].astype(str).values
s_te = test['f_27'].astype(str).values
all_full = pd.Series(np.concatenate([s_tr, s_te]))
vc_full = all_full.value_counts()
N_all = float(len(train) + len(test))
freq_full_te = pd.Series(s_te).map(vc_full).fillna(0).values.astype('float32') / N_all
rarity = 1.0 - freq_full_te
rarity_r = rank_norm(rarity)

# Seen/unseen masks via majority f_27 identity map
means = train.groupby('f_27')['target'].mean()
f27_map = (means >= 0.5).astype('int8').to_dict()
test_seen_mask = test['f_27'].isin(f27_map).values
test_unseen_mask = ~test_seen_mask
preds_seen = test.loc[test_seen_mask, 'f_27'].map(f27_map).astype('float32').values

alphas = [0.95, 0.97, 0.99]
for a in alphas:
    post_unseen = (a * rank_norm(base_pred) + (1 - a) * rarity_r).astype('float32')
    final = pd.read_csv('sample_submission.csv')[['id']].copy()
    final['target'] = np.zeros(len(final), dtype='float32')
    final.loc[test_seen_mask, 'target'] = preds_seen
    final.loc[test_unseen_mask, 'target'] = post_unseen[test_unseen_mask]
    final['target'] = final['target'].astype('float64').clip(0.0, 1.0)
    out_path = f'submission_unseen_rarity_a{int(round(a*100))}.csv'
    final.to_csv(out_path, index=False)
    log(f'Saved {out_path} (rarity rank post-process, alpha={a}).')

[2025-09-12 04:22:25] Applying rarity rank post-process on current TE-unseen blend (alpha in {0.95,0.97,0.99})...


[2025-09-12 04:22:29] Saved submission_unseen_rarity_a95.csv (rarity rank post-process, alpha=0.95).


[2025-09-12 04:22:29] Saved submission_unseen_rarity_a97.csv (rarity rank post-process, alpha=0.97).


[2025-09-12 04:22:29] Saved submission_unseen_rarity_a99.csv (rarity rank post-process, alpha=0.99).
